In [ ]:
import xml.etree.ElementTree as ET
import json
import re

def safe_float(text):
    """Convert a string to float after removing commas."""
    if text is None:
        return 0.0
    return float(re.sub(r",", "", text.strip()))

def parse_decision_logs(root, hero):
    """
    Parse the XML tree and, for every opponent decision (action) in rounds 1 and later
    (preflop and beyond), produce a snapshot log capturing the game state at the moment
    BEFORE the action is processed.
    
    In this snapshot:
      - The snapshot does NOT include any actions from round 0 (blinds/antes).
      - The re-indexed action counter is incremented for every action in rounds ≥ 1,
        so that no action gets a number of 0.
      - Only opponent actions are logged (the hero's actions are skipped).
      - The snapshot shows the acting opponent's hole cards (if available); the hero's
        or other players' hole cards are not revealed in the snapshot.
    """
    logs = []

    for game in root.findall('game'):
        general = game.find('general')
        if general is None:
            continue

        # --- Extract blinds and ante ---
        small_blind_elem = general.find('smallblind')
        big_blind_elem = general.find('bigblind')
        ante_elem = general.find('ante')
        small_blind = safe_float(small_blind_elem.text) if small_blind_elem is not None else 0.0
        big_blind = safe_float(big_blind_elem.text) if big_blind_elem is not None else 0.0
        ante = safe_float(ante_elem.text) if ante_elem is not None else 0.0

        # --- Process players ---
        players = {}        # key: player name, value: player info
        active_players = {} # tracks whether a player is still in the hand
        players_elem = general.find('players')
        if players_elem is None:
            continue

        for player_elem in players_elem.findall('player'):
            name = player_elem.attrib.get('name')
            seat = int(player_elem.attrib.get('seat'))
            is_dealer = (player_elem.attrib.get('dealer') == "1")
            chips = safe_float(player_elem.attrib.get('chips'))
            bet = safe_float(player_elem.attrib.get('bet'))
            win = safe_float(player_elem.attrib.get('win'))
            players[name] = {
                'seat': seat,
                'is_dealer': is_dealer,
                'chips': chips,
                'bet': bet,
                'win': win
            }
            active_players[name] = True

        # --- Determine relative positions ---
        # Sort players by seat number.
        sorted_players = sorted(players.items(), key=lambda item: item[1]['seat'])
        # Find the dealer (button). If none is marked, assume the first player is the dealer.
        dealer_index = None
        for i, (name, info) in enumerate(sorted_players):
            if info['is_dealer']:
                dealer_index = i
                break
        if dealer_index is None:
            dealer_index = 0

        button_player = sorted_players[dealer_index][0]
        small_blind_player = sorted_players[(dealer_index + 1) % len(sorted_players)][0]
        big_blind_player = sorted_players[(dealer_index + 2) % len(sorted_players)][0]

        # Assign relative position codes: small blind = 0, big blind = 1, button = 2, others = 3.
        player_positions = {}
        for name in players:
            if name == small_blind_player:
                players[name]['position_relative'] = 0
            elif name == big_blind_player:
                players[name]['position_relative'] = 1
            elif name == button_player:
                players[name]['position_relative'] = 2
            else:
                players[name]['position_relative'] = 3
            player_positions[name] = players[name]['position_relative']

        # Skip this game if the hero is not present.
        if hero not in players:
            continue

        # --- Initialize state variables ---
        pot_size = 0.0
        cumulative_actions = []  # will contain actions from rounds >= 1 only
        board_cards = []         # community cards seen so far
        pocket_cards = {}        # updated when encountering <cards type="Pocket">
        snapshot_action_counter = 0  # counter for actions in rounds >= 1

        # --- Process rounds in order (sorted by round number) ---
        rounds = game.findall('round')
        rounds = sorted(rounds, key=lambda r: int(r.attrib.get('no')))
        for r in rounds:
            round_no = int(r.attrib.get('no'))
            for elem in r:
                if elem.tag == 'cards':
                    card_type = elem.attrib.get('type')
                    if card_type == 'Pocket':
                        # Record pocket cards for the given player.
                        player = elem.attrib.get('player')
                        text = elem.text.strip() if elem.text else ""
                        cards = text.split()
                        # If cards are hidden (e.g., "X X"), record as unknown.
                        if any(card.upper().startswith("X") for card in cards):
                            pocket_cards[player] = ["unknown", "unknown"]
                        else:
                            pocket_cards[player] = cards
                    else:
                        # Community cards (flop, turn, river)
                        cards = elem.text.split() if elem.text else []
                        board_cards.extend(cards)
                elif elem.tag == 'action':
                    # Build an action dictionary.
                    action_details = {
                        'round': round_no,
                        'player': elem.attrib.get('player'),
                        'action_type': int(elem.attrib.get('type')),
                        'action_sum': safe_float(elem.attrib.get('sum'))
                    }

                    # For actions in round 0 (blinds/antes), update state only (do not record them).
                    if round_no < 1:
                        pot_size += action_details['action_sum']
                        # (If desired, you could update pocket_cards from round 0 as well.)
                        continue

                    # For actions in rounds >= 1, first assign a new re-indexed action number.
                    snapshot_action_counter += 1
                    # Copy the action_details and add the new action number.
                    action_details['action_no'] = snapshot_action_counter

                    # If the acting player is not the hero, produce a snapshot BEFORE processing the action.
                    # (The snapshot’s "previous_actions" will be the cumulative_actions so far.)
                    if action_details['player'] != hero:
                        is_button = (action_details['player'] == button_player)
                        actor_cards = pocket_cards.get(action_details['player'], ["unknown", "unknown"])
                        # Build the snapshot.
                        snapshot = {
                            "gamecode": game.attrib.get("gamecode"),
                            "round_no": round_no,
                            "current_street": (
                                "preflop" if round_no == 1 else
                                "flop" if round_no == 2 else
                                "turn" if round_no == 3 else
                                "river" if round_no == 4 else "unknown"
                            ),
                            "blinds": {
                                "small_blind": small_blind,
                                "big_blind": big_blind,
                                "ante": ante
                            },
                            "player_positions": player_positions,
                            "player_stacks": {name: players[name]['chips'] for name in players},
                            "pot_size": pot_size,  # state BEFORE processing this action
                            "board_cards": board_cards.copy(),
                            "previous_actions": cumulative_actions.copy(),  # only round>=1 actions so far
                            "action": action_details.copy(),
                            "players_remaining": sum(1 for active in active_players.values() if active),
                            "is_button": is_button,
                            "actor_hole_cards": pocket_cards.get(action_details['player'], ["unknown", "unknown"])
                        }
                        logs.append(snapshot)

                    # Now update the state: add this action to the cumulative history.
                    cumulative_actions.append(action_details)
                    pot_size += action_details['action_sum']
                    # Mark the actor as inactive if the action is a fold (we assume type==0 means fold).
                    if action_details['action_type'] == 0:
                        active_players[action_details['player']] = False

    return logs

if __name__ == '__main__':
    try:
        # Parse the XML file (adjust 'test.xml' to your filename)
        tree = ET.parse('test.xml')
        root = tree.getroot()

        # Determine the hero's nickname from the session-level <nickname> element.
        session_general = root.find('general')
        if session_general is not None and session_general.find('nickname') is not None:
            hero = session_general.find('nickname').text.strip()
        else:
            hero = "mmpq8j6x8"  # fallback if not found

        # Parse opponent decision snapshots (skipping hero actions and round 0 actions)
        logs = parse_decision_logs(root, hero)
        print(json.dumps(logs, indent=4))
    except ET.ParseError as e:
        print("Error parsing XML file:", e)


## Improve version of extract data from single iPoker HH xml file

In [2]:
#!/usr/bin/env python3
import xml.etree.ElementTree as ET
import json, re

def safe_float(text):
    try:
        # remove any commas or other non-digit/dot characters
        return float(re.sub(r"[^\d\.]", "", text))
    except Exception:
        return 0.0

def street_from_round(round_no):
    return {1: "preflop", 2: "flop", 3: "turn", 4: "river"}.get(round_no, "unknown")

def get_hole_cards(game, player):
    # Search all rounds for a <cards type="Pocket" player="..."> element.
    # If the text contains "X" (hidden cards) we return ["unknown","unknown"].
    for r in game.findall('round'):
        for elem in r.findall('cards'):
            if elem.attrib.get("type") == "Pocket" and elem.attrib.get("player") == player:
                if elem.text:
                    cards = elem.text.split()
                    if any(c.upper().startswith("X") for c in cards):
                        return ["unknown", "unknown"]
                    else:
                        return cards
    return ["unknown", "unknown"]

def parse_decision_logs(root, hero):
    logs = []
    # Process every game
    for game in root.findall('game'):
        gamecode = game.attrib.get("gamecode", "")
        general = game.find('general')
        # extract blinds and ante (if any)
        blinds = {}
        blinds['small_blind'] = safe_float(general.findtext('smallblind', default="0"))
        blinds['big_blind'] = safe_float(general.findtext('bigblind', default="0"))
        blinds['ante'] = safe_float(general.findtext('ante', default="0"))
        
        # Process players. We assume the players are listed in a <players> element.
        players_elem = general.find('players')
        player_positions = {}   # will hold each player’s relative position code
        player_stacks = {}      # the chip stacks at the beginning of the hand
        # sort players by their seat number
        sorted_players = sorted(players_elem.findall('player'), key=lambda p: int(p.attrib.get('seat', 0)))
        names = [p.attrib['name'] for p in sorted_players]
        # determine the dealer: use the one whose dealer attribute equals "1" (or if none, the first one)
        dealer = None
        for p in sorted_players:
            if p.attrib.get('dealer','0')=='1':
                dealer = p.attrib['name']
                break
        if dealer is None:
            dealer = names[0]
        # Create an “order” starting with the player immediately after the dealer (wrapping around).
        dealer_index = names.index(dealer)
        order = names[dealer_index+1:] + names[:dealer_index+1]
        # For instance, assign positions 0,1,...,n-1 in the order determined.
        for i, name in enumerate(order):
            player_positions[name] = i
        for p in sorted_players:
            name = p.attrib['name']
            player_stacks[name] = safe_float(p.attrib.get('chips', "0"))
        
        # All players start active.
        active_players = {name: True for name in player_positions}
        pot_size = 0.0
        board_cards = []
        cumulative_actions = []  # history of actions (only rounds >= 1)
        snapshot_action_counter = 0

        # Process each round in the game.
        for r in game.findall('round'):
            round_no = int(r.attrib.get('no', 0))
            for child in r:
                if child.tag == "cards":
                    # If it is not a Pocket card then we assume community cards.
                    if child.attrib.get("type") != "Pocket":
                        if child.text:
                            board_cards.extend(child.text.split())
                elif child.tag == "action":
                    action_details = {
                        'round': round_no,
                        'player': child.attrib.get('player'),
                        'action_type': int(child.attrib.get('type')),
                        'action_sum': safe_float(child.attrib.get('sum'))
                    }
                    # Skip round 0 actions: update pot only.
                    if round_no < 1:
                        pot_size += action_details['action_sum']
                        continue

                    # For rounds >=1: assign a re-indexed action number.
                    snapshot_action_counter += 1
                    action_details['action_no'] = snapshot_action_counter

                    # For opponent actions (non–hero) produce a snapshot BEFORE processing the action.
                    if child.attrib.get('player') != hero:
                        snapshot = {
                            "gamecode": gamecode,
                            "round_no": round_no,
                            "current_street": street_from_round(round_no),
                            "blinds": blinds,
                            "player_positions": player_positions,
                            "player_stacks": player_stacks,
                            "pot_size": pot_size,  # pot _before_ current action
                            "board_cards": board_cards.copy(),
                            "previous_actions": [],
                            "action": action_details.copy(),
                            "players_remaining": sum(1 for v in active_players.values() if v),
                            "is_button": (player_positions.get(child.attrib.get('player')) == len(player_positions)-1),
                            "actor_hole_cards": get_hole_cards(game, child.attrib.get('player')),
                            # NEW FIELDS:
                            "actor_stack_size": player_stacks.get(child.attrib.get('player'), 0.0),
                            "actor_position": player_positions.get(child.attrib.get('player'))
                        }
                        # Add only previous actions that have been recorded (all from rounds>=1).
                        for act in cumulative_actions:
                            act2 = act.copy()
                            act2["player_position"] = player_positions.get(act["player"])
                            snapshot["previous_actions"].append(act2)
                        logs.append(snapshot)
                    
                    # Update running state: add this action to cumulative_actions and update pot.
                    cumulative_actions.append(action_details.copy())
                    pot_size += action_details['action_sum']
                    # If the action is a fold (action_type == 0) then mark the player as inactive.
                    if action_details['action_type'] == 0:
                        active_players[action_details['player']] = False
        # end of game rounds
    return logs

if __name__ == '__main__':
    try:
        tree = ET.parse('test.xml')
        root = tree.getroot()
        # Get the hero name from the session <nickname> element.
        session_general = root.find('general')
        if session_general is not None and session_general.find('nickname') is not None:
            hero = session_general.find('nickname').text.strip()
        else:
            hero = "mmpq8j6x8"  # fallback
        logs = parse_decision_logs(root, hero)
        print(json.dumps(logs, indent=4))
    except ET.ParseError as e:
        print("Error parsing XML file:", e)


[
    {
        "gamecode": "10136070425",
        "round_no": 1,
        "current_street": "preflop",
        "blinds": {
            "small_blind": 10.0,
            "big_blind": 20.0,
            "ante": 0.0
        },
        "player_positions": {
            "GGM3M": 0,
            "Bekocabron": 1,
            "mmpq8j6x8": 2
        },
        "player_stacks": {
            "GGM3M": 500.0,
            "Bekocabron": 500.0,
            "mmpq8j6x8": 500.0
        },
        "pot_size": 30.0,
        "board_cards": [],
        "previous_actions": [
            {
                "round": 1,
                "player": "mmpq8j6x8",
                "action_type": 0,
                "action_sum": 0.0,
                "action_no": 1,
                "player_position": 2
            }
        ],
        "action": {
            "round": 1,
            "player": "GGM3M",
            "action_type": 3,
            "action_sum": 10.0,
            "action_no": 2
        },
        "players_remainin

## Extract iPoker HH from single hand history file(Ready for training model)

In [ ]:
#!/usr/bin/env python3
import xml.etree.ElementTree as ET
import json, re

def safe_float(text):
    try:
        # remove any commas or other non-digit/dot characters
        return float(re.sub(r"[^\d\.]", "", text))
    except Exception:
        return 0.0

def street_from_round(round_no):
    return {1: "preflop", 2: "flop", 3: "turn", 4: "river"}.get(round_no, "unknown")

def get_hole_cards(game, player):
    # Search all rounds for a <cards type="Pocket" player="..."> element.
    # If the text contains "X" (hidden cards) we return ["unknown","unknown"].
    for r in game.findall('round'):
        for elem in r.findall('cards'):
            if elem.attrib.get("type") == "Pocket" and elem.attrib.get("player") == player:
                if elem.text:
                    cards = elem.text.split()
                    if any(c.upper().startswith("X") for c in cards):
                        return ["unknown", "unknown"]
                    else:
                        return cards
    return ["unknown", "unknown"]

def parse_decision_logs(root, hero):
    logs = []
    # Process every game
    for game in root.findall('game'):
        gamecode = game.attrib.get("gamecode", "")
        general = game.find('general')
        # extract blinds and ante (if any)
        blinds = {}
        blinds['small_blind'] = safe_float(general.findtext('smallblind', default="0"))
        blinds['big_blind'] = safe_float(general.findtext('bigblind', default="0"))
        blinds['ante'] = safe_float(general.findtext('ante', default="0"))
        
        # Process players. We assume the players are listed in a <players> element.
        players_elem = general.find('players')
        player_positions = {}   # will hold each player’s relative position code
        player_stacks = {}      # the chip stacks at the beginning of the hand
        # sort players by their seat number
        sorted_players = sorted(players_elem.findall('player'), key=lambda p: int(p.attrib.get('seat', 0)))
        names = [p.attrib['name'] for p in sorted_players]
        # determine the dealer: use the one whose dealer attribute equals "1" (or if none, the first one)
        dealer = None
        for p in sorted_players:
            if p.attrib.get('dealer','0')=='1':
                dealer = p.attrib['name']
                break
        if dealer is None:
            dealer = names[0]
        # Create an “order” starting with the player immediately after the dealer (wrapping around).
        dealer_index = names.index(dealer)
        order = names[dealer_index+1:] + names[:dealer_index+1]
        # For instance, assign positions 0,1,...,n-1 in the order determined.
        for i, name in enumerate(order):
            player_positions[name] = i
        for p in sorted_players:
            name = p.attrib['name']
            player_stacks[name] = safe_float(p.attrib.get('chips', "0"))
        
        # All players start active.
        active_players = {name: True for name in player_positions}
        pot_size = 0.0
        board_cards = []
        cumulative_actions = []  # history of actions (only rounds >= 1)
        snapshot_action_counter = 0

        # Process each round in the game.
        for r in game.findall('round'):
            round_no = int(r.attrib.get('no', 0))
            for child in r:
                if child.tag == "cards":
                    # If it is not a Pocket card then we assume community cards.
                    if child.attrib.get("type") != "Pocket":
                        if child.text:
                            board_cards.extend(child.text.split())
                elif child.tag == "action":
                    action_details = {
                        'round': round_no,
                        'player': child.attrib.get('player'),
                        'action_type': int(child.attrib.get('type')),
                        'action_sum': safe_float(child.attrib.get('sum'))
                    }
                    # Skip round 0 actions: update pot only.
                    if round_no < 1:
                        pot_size += action_details['action_sum']
                        continue

                    # For rounds >=1: assign a re-indexed action number.
                    snapshot_action_counter += 1
                    action_details['action_no'] = snapshot_action_counter

                    # For opponent actions (non–hero) produce a snapshot BEFORE processing the action.
                    if child.attrib.get('player') != hero:
                        snapshot = {
                            "gamecode": gamecode,
                            "round_no": round_no,
                            "current_street": street_from_round(round_no),
                            "blinds": blinds,
                            "player_positions": player_positions,
                            "player_stacks": player_stacks,
                            "pot_size": pot_size,  # pot _before_ current action
                            "board_cards": board_cards.copy(),
                            "previous_actions": [],
                            "action": action_details.copy(),
                            "players_remaining": sum(1 for v in active_players.values() if v),
                            "is_button": (player_positions.get(child.attrib.get('player')) == len(player_positions)-1),
                            "actor_hole_cards": get_hole_cards(game, child.attrib.get('player')),
                            # NEW FIELDS:
                            "actor_stack_size": player_stacks.get(child.attrib.get('player'), 0.0),
                            "actor_position": player_positions.get(child.attrib.get('player'))
                        }
                        # Add only previous actions that have been recorded (all from rounds>=1).
                        for act in cumulative_actions:
                            act2 = act.copy()
                            act2["player_position"] = player_positions.get(act["player"])
                            snapshot["previous_actions"].append(act2)
                        logs.append(snapshot)
                    
                    # Update running state: add this action to cumulative_actions and update pot.
                    cumulative_actions.append(action_details.copy())
                    pot_size += action_details['action_sum']
                    # If the action is a fold (action_type == 0) then mark the player as inactive.
                    if action_details['action_type'] == 0:
                        active_players[action_details['player']] = False
        # end of game rounds
    return logs

if __name__ == '__main__':
    try:
        tree = ET.parse('test.xml')
        root = tree.getroot()
        # Get the hero name from the session <nickname> element.
        session_general = root.find('general')
        if session_general is not None and session_general.find('nickname') is not None:
            hero = session_general.find('nickname').text.strip()
        else:
            hero = "mmpq8j6x8"  # fallback
        logs = parse_decision_logs(root, hero)
        # Instead of printing, write to a JSON file:
        with open('test_hand.json', 'w') as outfile:
            json.dump(logs, outfile, indent=4)
        print("Data extraction complete. Saved to test_hand.json")
    except ET.ParseError as e:
        print("Error parsing XML file:", e)


Data extraction complete. Saved to logs.json


## Extract iPoker HH from folder

In [6]:
#!/usr/bin/env python3
import os
import xml.etree.ElementTree as ET
import json
import re

# ------------------------------
# Helper functions
# ------------------------------

def safe_float(text):
    try:
        return float(re.sub(r"[^\d\.]", "", text))
    except Exception:
        return 0.0

def street_from_round(round_no):
    return {1: "preflop", 2: "flop", 3: "turn", 4: "river"}.get(round_no, "unknown")

def get_hole_cards(game, player):
    for r in game.findall('round'):
        for elem in r.findall('cards'):
            if elem.attrib.get("type") == "Pocket" and elem.attrib.get("player") == player:
                if elem.text:
                    cards = elem.text.split()
                    if any(c.upper().startswith("X") for c in cards):
                        return ["unknown", "unknown"]
                    else:
                        return cards
    return ["unknown", "unknown"]

def simplify_action(action_details, round_no, blinds, pot_before_action):
    """
    Simplify the action into a single action_type string.
    
    Allowed original types:
      - 0 -> fold
      - 3 -> call
      - 4 -> check
      - 5, 7, 23 -> raise
         * Preflop (round 1): using big blind as reference.
             2.0×BB to 2.3×BB → small raise preflop
             2.3×BB to 3.0×BB → mid raise preflop
             3.0×BB to 4.0×BB → big raise preflop
             ≥ 4.0×BB         → all in preflop
         * Postflop (round > 1): using the pot (before the action) as reference.
             < 35% of pot      → small raise postflop
             35% to 70% of pot  → mid raise postflop
             70% to 110% of pot → big raise postflop
             ≥ 110% of pot     → all in postflop
    """
    allowed_types = {0: "fold", 3: "call", 4: "check", 5: "raise", 7: "raise", 23: "raise"}
    orig_type = action_details['action_type']
    if orig_type not in allowed_types:
        return None
    base_action = allowed_types[orig_type]
    new_action = action_details.copy()
    if base_action != "raise":
        new_action["simple_action_type"] = base_action
    else:
        if round_no == 1:
            bb = blinds.get("big_blind", 1)
            ratio = action_details['action_sum'] / bb if bb != 0 else 0
            if 2.0 <= ratio < 2.3:
                new_action["simple_action_type"] = "small raise preflop"
            elif 2.3 <= ratio < 3.0:
                new_action["simple_action_type"] = "mid raise preflop"
            elif 3.0 <= ratio < 4.0:
                new_action["simple_action_type"] = "big raise preflop"
            elif ratio >= 4.0:
                new_action["simple_action_type"] = "all in preflop"
            else:
                new_action["simple_action_type"] = "raise"
        else:
            ratio = action_details['action_sum'] / pot_before_action if pot_before_action > 0 else 0
            if ratio < 0.35:
                new_action["simple_action_type"] = "small raise postflop"
            elif ratio < 0.70:
                new_action["simple_action_type"] = "mid raise postflop"
            elif ratio < 1.10:
                new_action["simple_action_type"] = "big raise postflop"
            else:
                new_action["simple_action_type"] = "all in postflop"
    return new_action

def parse_decision_logs(root, hero):
    logs = []
    for game in root.findall('game'):
        gamecode = game.attrib.get("gamecode", "")
        general = game.find('general')
        blinds = {
            'small_blind': safe_float(general.findtext('smallblind', default="0")),
            'big_blind': safe_float(general.findtext('bigblind', default="0")),
            'ante': safe_float(general.findtext('ante', default="0"))
        }
        players_elem = general.find('players')
        player_positions = {}
        # Store initial stacks from the XML.
        player_stacks = {}
        sorted_players = sorted(players_elem.findall('player'), key=lambda p: int(p.attrib.get('seat', 0)))
        names = [p.attrib['name'] for p in sorted_players]
        dealer = None
        for p in sorted_players:
            if p.attrib.get('dealer', '0') == '1':
                dealer = p.attrib['name']
                break
        if dealer is None:
            dealer = names[0]
        dealer_index = names.index(dealer)
        order = names[dealer_index+1:] + names[:dealer_index+1]
        for i, name in enumerate(order):
            player_positions[name] = i
        for p in sorted_players:
            name = p.attrib['name']
            player_stacks[name] = safe_float(p.attrib.get('chips', "0"))
        
        active_players = {name: True for name in player_positions}
        pot_size = 0.0
        board_cards = []
        cumulative_actions = []  # Simplified actions list (each now includes "action_round")
        snapshot_action_counter = 0
        
        # Track contributions from each player (blinds, bets, etc.)
        player_contributions = {name: 0.0 for name in player_positions}
        
        for r in game.findall('round'):
            round_no = int(r.attrib.get('no', 0))
            for child in r:
                if child.tag == "cards":
                    if child.attrib.get("type") != "Pocket":
                        if child.text:
                            board_cards.extend(child.text.split())
                elif child.tag == "action":
                    action_details = {
                        'player': child.attrib.get('player'),
                        'action_type': int(child.attrib.get('type')),
                        'action_sum': safe_float(child.attrib.get('sum')),
                        'action_round': round_no
                    }
                    player = action_details['player']
                    contribution = action_details['action_sum']
                    
                    # For round 0 (blinds/antes) update contributions and pot; no snapshot is created.
                    if round_no < 1:
                        player_contributions[player] += contribution
                        pot_size += contribution
                        continue
                        
                    # For rounds ≥ 1, first capture the current pot (before the current action is added)
                    current_pot = pot_size
                    # Actor's current stack = initial stack minus chips already committed.
                    actor_current_stack = player_stacks.get(player, 0.0) - player_contributions.get(player, 0.0)
                    
                    # Simplify action into a single string category.
                    simple_action = simplify_action(action_details, round_no, blinds, current_pot)
                    
                    if simple_action is not None:
                        snapshot_action_counter += 1
                        simple_action['action_no'] = snapshot_action_counter
                        
                        # Compute current stack for every player.
                        current_player_stacks = {p: player_stacks[p] - player_contributions.get(p, 0.0)
                                                 for p in player_stacks}
                        
                        # Create snapshot only for non-hero actions.
                        if player != hero:
                            snapshot = {
                                "gamecode": gamecode,
                                "round_no": round_no,
                                "current_street": street_from_round(round_no),
                                "blinds": blinds,
                                "player_positions": player_positions,
                                "player_stacks": current_player_stacks,
                                "pot_size": current_pot,
                                "board_cards": board_cards.copy(),
                                "previous_actions": [],
                                "action": simple_action.copy(),
                                "players_remaining": sum(1 for v in active_players.values() if v),
                                "is_button": (player_positions.get(player) == len(player_positions)-1),
                                "actor_hole_cards": get_hole_cards(game, player),
                                "actor_stack_size": actor_current_stack,
                                "actor_position": player_positions.get(player)
                            }
                            for act in cumulative_actions:
                                act2 = act.copy()
                                act2["player_position"] = player_positions.get(act["player"])
                                snapshot["previous_actions"].append(act2)
                            logs.append(snapshot)
                    
                    # Now update contributions and pot AFTER snapshot creation.
                    player_contributions[player] += contribution
                    pot_size += contribution
                    
                    if simple_action is not None:
                        cumulative_actions.append(simple_action)
                    
                    # Mark player as inactive if they folded.
                    if action_details.get('action_type') == 0:
                        active_players[player] = False
    return logs

# ------------------------------
# Process All XML Files in ipoker_hh Folder
# ------------------------------

def process_all_hand_history(root_folder, hero=None):
    all_logs = []
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith(".xml"):
                file_path = os.path.join(dirpath, filename)
                try:
                    tree = ET.parse(file_path)
                    root_xml = tree.getroot()
                    if hero is None:
                        session_general = root_xml.find('general')
                        if session_general is not None and session_general.find('nickname') is not None:
                            hero = session_general.find('nickname').text.strip()
                    logs = parse_decision_logs(root_xml, hero)
                    all_logs.extend(logs)
                    print(f"Processed file: {file_path} -> {len(logs)} snapshots.")
                except ET.ParseError as e:
                    print(f"Error parsing XML file: {file_path}", e)
    return all_logs

# ------------------------------
# Main
# ------------------------------

if __name__ == '__main__':
    root_folder = "ipoker_hh_test"  # Adjust the folder path as needed
    hero_name = None  # Or manually set your hero's name
    print("Processing all hand history XML files in folder:", root_folder)
    all_logs = process_all_hand_history(root_folder, hero=hero_name)
    print("Total snapshots extracted:", len(all_logs))
    with open("logs.json", "w") as outfile:
        json.dump(all_logs, outfile, indent=4)
    print("Data extraction complete. Saved to logs.json")


Processing all hand history XML files in folder: ipoker_hh_test
Processed file: ipoker_hh_test/Tournaments/7733651957.xml -> 38 snapshots.
Processed file: ipoker_hh_test/Tournaments/4980566349.xml -> 31 snapshots.
Processed file: ipoker_hh_test/Tournaments/4978995976.xml -> 86 snapshots.
Processed file: ipoker_hh_test/Tournaments/7736176979.xml -> 38 snapshots.
Processed file: ipoker_hh_test/Tournaments/4978983872.xml -> 80 snapshots.
Processed file: ipoker_hh_test/Tournaments/4968024165.xml -> 51 snapshots.
Processed file: ipoker_hh_test/Tournaments/4977840702.xml -> 59 snapshots.
Processed file: ipoker_hh_test/Tournaments/4973790181.xml -> 85 snapshots.
Processed file: ipoker_hh_test/Tournaments/4978600192.xml -> 53 snapshots.
Processed file: ipoker_hh_test/Tournaments/4979832638.xml -> 100 snapshots.
Processed file: ipoker_hh_test/Tournaments/4977072864.xml -> 66 snapshots.
Processed file: ipoker_hh_test/Tournaments/4976476504.xml -> 75 snapshots.
Processed file: ipoker_hh_test/Tour

## Extract data from hand history folder, make the player name into number

In [10]:
#!/usr/bin/env python3
import os
import xml.etree.ElementTree as ET
import json
import re

# ------------------------------
# Helper functions
# ------------------------------

def safe_float(text):
    try:
        return float(re.sub(r"[^\d\.]", "", text))
    except Exception:
        return 0.0

def street_from_round(round_no):
    return {1: "preflop", 2: "flop", 3: "turn", 4: "river"}.get(round_no, "unknown")

def get_hole_cards(game, player):
    for r in game.findall('round'):
        for elem in r.findall('cards'):
            if elem.attrib.get("type") == "Pocket" and elem.attrib.get("player") == player:
                if elem.text:
                    cards = elem.text.split()
                    if any(c.upper().startswith("X") for c in cards):
                        return ["unknown", "unknown"]
                    else:
                        return cards
    return ["unknown", "unknown"]

def simplify_action(action_details, round_no, blinds, pot_before_action):
    """
    Simplify the action into a single string category.
    
    Allowed original types:
      - 0 → fold
      - 3 → call
      - 4 → check
      - 5, 7, 23 → raise, which are further categorized as follows:
          * Preflop (round 1), using the big blind as reference:
              2.0×BB to 2.3×BB → "small raise preflop"
              2.3×BB to 3.0×BB → "mid raise preflop"
              3.0×BB to 4.0×BB → "big raise preflop"
              ≥ 4.0×BB         → "all in preflop"
          * Postflop (round > 1), using the pot (before the action) as reference:
              < 35% of pot      → "small raise postflop"
              35% to 70% of pot  → "mid raise postflop"
              70% to 110% of pot → "big raise postflop"
              ≥ 110% of pot     → "all in postflop"
    """
    allowed_types = {0: "fold", 3: "call", 4: "check", 5: "raise", 7: "raise", 23: "raise"}
    orig_type = action_details['action_type']
    if orig_type not in allowed_types:
        return None
    base_action = allowed_types[orig_type]
    new_action = action_details.copy()
    if base_action != "raise":
        new_action["simple_action_type"] = base_action
    else:
        if round_no == 1:
            bb = blinds.get("big_blind", 1)
            ratio = action_details['action_sum'] / bb if bb != 0 else 0
            if 2.0 <= ratio < 2.3:
                new_action["simple_action_type"] = "small raise preflop"
            elif 2.3 <= ratio < 3.0:
                new_action["simple_action_type"] = "mid raise preflop"
            elif 3.0 <= ratio < 4.0:
                new_action["simple_action_type"] = "big raise preflop"
            elif ratio >= 4.0:
                new_action["simple_action_type"] = "all in preflop"
            else:
                new_action["simple_action_type"] = "raise"
        else:
            ratio = action_details['action_sum'] / pot_before_action if pot_before_action > 0 else 0
            if ratio < 0.35:
                new_action["simple_action_type"] = "small raise postflop"
            elif ratio < 0.70:
                new_action["simple_action_type"] = "mid raise postflop"
            elif ratio < 1.10:
                new_action["simple_action_type"] = "big raise postflop"
            else:
                new_action["simple_action_type"] = "all in postflop"
    return new_action

def parse_decision_logs(root, hero):
    logs = []
    for game in root.findall('game'):
        gamecode = game.attrib.get("gamecode", "")
        general = game.find('general')
        blinds = {
            'small_blind': safe_float(general.findtext('smallblind', default="0")),
            'big_blind': safe_float(general.findtext('bigblind', default="0")),
            'ante': safe_float(general.findtext('ante', default="0"))
        }
        
        # Get players element and determine initial stacks.
        players_elem = general.find('players')
        # Identify small blind and big blind using round 0 actions.
        small_blind_player = None
        big_blind_player = None
        for r in game.findall('round'):
            if int(r.attrib.get('no', 0)) == 0:
                for child in r:
                    if child.tag == "action":
                        if child.attrib.get('type') == "1":
                            small_blind_player = child.attrib.get('player')
                        elif child.attrib.get('type') == "2":
                            big_blind_player = child.attrib.get('player')
                break
        # Identify the button player from the dealer flag.
        button_player = None
        for p in players_elem.findall('player'):
            if p.attrib.get('dealer','0') == '1':
                button_player = p.attrib.get('name')
                break
        
        # Create mapping: small blind → 0, big blind → 1, button → 2.
        player_positions = {}
        if small_blind_player is not None:
            player_positions[small_blind_player] = 0
        if big_blind_player is not None:
            player_positions[big_blind_player] = 1
        if button_player is not None:
            player_positions[button_player] = 2
        
        # Get initial stacks for the mapped players.
        player_stacks = {}
        for p in players_elem.findall('player'):
            name = p.attrib['name']
            if name in player_positions:
                player_stacks[name] = safe_float(p.attrib.get('chips', "0"))
        
        active_players = {name: True for name in player_positions}
        pot_size = 0.0
        board_cards = []
        cumulative_actions = []  # simplified actions (each includes "action_round")
        snapshot_action_counter = 0
        
        # Track cumulative contributions from each player.
        player_contributions = {name: 0.0 for name in player_positions}
        
        for r in game.findall('round'):
            round_no = int(r.attrib.get('no', 0))
            for child in r:
                if child.tag == "cards":
                    if child.attrib.get("type") != "Pocket":
                        if child.text:
                            board_cards.extend(child.text.split())
                elif child.tag == "action":
                    action_details = {
                        'player': child.attrib.get('player'),
                        'action_type': int(child.attrib.get('type')),
                        'action_sum': safe_float(child.attrib.get('sum')),
                        'action_round': round_no
                    }
                    player = action_details['player']
                    contribution = action_details['action_sum']
                    
                    # For round 0 (blinds/antes): update contributions and pot; no snapshot.
                    if round_no < 1:
                        if player in player_contributions:
                            player_contributions[player] += contribution
                        pot_size += contribution
                        continue
                        
                    # For rounds ≥ 1, capture current pot (before adding current action).
                    current_pot = pot_size
                    # Actor's current stack (as seen when taking the action).
                    actor_current_stack = player_stacks.get(player, 0.0) - player_contributions.get(player, 0.0)
                    
                    # Simplify the action.
                    simple_action = simplify_action(action_details, round_no, blinds, current_pot)
                    
                    if simple_action is not None:
                        snapshot_action_counter += 1
                        simple_action['action_no'] = snapshot_action_counter
                        # Replace the "player" field with its numeric value.
                        numeric_player = player_positions.get(player)
                        simple_action["player"] = numeric_player
                        
                        # Build current stacks keyed by numeric positions.
                        current_player_stacks = { player_positions[name] : (player_stacks[name] - player_contributions.get(name, 0.0))
                                                 for name in player_positions }
                        
                        # Create snapshot only for non-hero actions.
                        if player != hero:
                            snapshot = {
                                "gamecode": gamecode,
                                "round_no": round_no,
                                "current_street": street_from_round(round_no),
                                "blinds": blinds,
                                "player_positions": player_positions,  # mapping from name to number
                                "player_stacks": current_player_stacks,  # mapping from number to current stack
                                "pot_size": current_pot,
                                "board_cards": board_cards.copy(),
                                "previous_actions": [],
                                "action": simple_action.copy(),
                                "players_remaining": sum(1 for v in active_players.values() if v),
                                "is_button": (player_positions.get(player) == 2),
                                "actor_hole_cards": get_hole_cards(game, player),
                                "actor_stack_size": actor_current_stack,
                                "actor_position": player_positions.get(player)
                            }
                            # For previous actions, use the numeric "player" already set.
                            for act in cumulative_actions:
                                act2 = act.copy()
                                # Since act["player"] is now numeric, we simply assign it.
                                act2["player_position"] = act["player"]
                                snapshot["previous_actions"].append(act2)
                            logs.append(snapshot)
                    
                    # Update contributions and pot AFTER snapshot creation.
                    if player in player_contributions:
                        player_contributions[player] += contribution
                    pot_size += contribution
                    
                    if simple_action is not None:
                        cumulative_actions.append(simple_action)
                    
                    # Mark a player as inactive if they folded.
                    if action_details.get('action_type') == 0:
                        active_players[player] = False
    return logs

# ------------------------------
# Process All XML Files in ipoker_hh Folder
# ------------------------------

def process_all_hand_history(root_folder, hero=None):
    all_logs = []
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith(".xml"):
                file_path = os.path.join(dirpath, filename)
                try:
                    tree = ET.parse(file_path)
                    root_xml = tree.getroot()
                    if hero is None:
                        session_general = root_xml.find('general')
                        if session_general is not None and session_general.find('nickname') is not None:
                            hero = session_general.find('nickname').text.strip()
                    logs = parse_decision_logs(root_xml, hero)
                    all_logs.extend(logs)
                    print(f"Processed file: {file_path} -> {len(logs)} snapshots.")
                except ET.ParseError as e:
                    print(f"Error parsing XML file: {file_path}", e)
    return all_logs

# ------------------------------
# Main
# ------------------------------

if __name__ == '__main__':
    root_folder = "ipoker_hh_test"  # Adjust folder path as needed.
    hero_name = None  # Or manually set your hero's name.
    print("Processing all hand history XML files in folder:", root_folder)
    all_logs = process_all_hand_history(root_folder, hero=hero_name)
    print("Total snapshots extracted:", len(all_logs))
    with open("logs.json", "w") as outfile:
        json.dump(all_logs, outfile, indent=4)
    print("Data extraction complete. Saved to logs.json")


Processing all hand history XML files in folder: ipoker_hh_test
Processed file: ipoker_hh_test/Tournaments/7733651957.xml -> 38 snapshots.
Processed file: ipoker_hh_test/Tournaments/4980566349.xml -> 31 snapshots.
Processed file: ipoker_hh_test/Tournaments/4978995976.xml -> 86 snapshots.
Processed file: ipoker_hh_test/Tournaments/7736176979.xml -> 38 snapshots.
Processed file: ipoker_hh_test/Tournaments/4978983872.xml -> 80 snapshots.
Processed file: ipoker_hh_test/Tournaments/4968024165.xml -> 51 snapshots.
Processed file: ipoker_hh_test/Tournaments/4977840702.xml -> 59 snapshots.
Processed file: ipoker_hh_test/Tournaments/4973790181.xml -> 85 snapshots.
Processed file: ipoker_hh_test/Tournaments/4978600192.xml -> 53 snapshots.
Processed file: ipoker_hh_test/Tournaments/4979832638.xml -> 100 snapshots.
Processed file: ipoker_hh_test/Tournaments/4977072864.xml -> 66 snapshots.
Processed file: ipoker_hh_test/Tournaments/4976476504.xml -> 75 snapshots.
Processed file: ipoker_hh_test/Tour

## Exam the output json file

In [11]:
import json

with open('logs.json', 'r') as f:
    data = json.load(f)
print("Number of snapshots:", len(data))
print("First snapshot:")
print(json.dumps(data[0], indent=4))
print(json.dumps(data[1], indent=4))
print(json.dumps(data[2], indent=4))
print(json.dumps(data[3], indent=4))
print(json.dumps(data[4], indent=4))


Number of snapshots: 431736
First snapshot:
{
    "gamecode": "10144138013",
    "round_no": 1,
    "current_street": "preflop",
    "blinds": {
        "small_blind": 10.0,
        "big_blind": 20.0,
        "ante": 0.0
    },
    "player_positions": {
        "mcsances1": 0,
        "mmpq8j6x8": 1,
        "Natarehs": 2
    },
    "player_stacks": {
        "0": 490.0,
        "1": 480.0,
        "2": 500.0
    },
    "pot_size": 30.0,
    "board_cards": [],
    "previous_actions": [],
    "action": {
        "player": 2,
        "action_type": 23,
        "action_sum": 40.0,
        "action_round": 1,
        "simple_action_type": "small raise preflop",
        "action_no": 1
    },
    "players_remaining": 3,
    "is_button": true,
    "actor_hole_cards": [
        "unknown",
        "unknown"
    ],
    "actor_stack_size": 500.0,
    "actor_position": 2
}
{
    "gamecode": "10144138013",
    "round_no": 1,
    "current_street": "preflop",
    "blinds": {
        "small_blind": 10.

## Model training

In [5]:
#!/usr/bin/env python3
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, LSTM, Embedding, Concatenate, Dropout
)
from tensorflow.keras.optimizers import Adam

# =============================================================================
# 1. Helper Functions for Feature Encoding
# =============================================================================

def one_hot_round(round_no):
    """One-hot encode round number (1=preflop, 2=flop, 3=turn, 4=river)."""
    vec = np.zeros(4)
    if 1 <= round_no <= 4:
        vec[round_no - 1] = 1
    return vec

def one_hot_position(pos, max_players=10):
    """One-hot encode a player's position (an integer in [0, max_players-1])."""
    vec = np.zeros(max_players)
    if pos < max_players:
        vec[pos] = 1
    return vec

def card_to_onehot(card):
    """
    Convert a card string (e.g., 'S4', 'HA', 'C10') to a 52-dim one-hot vector.
    If the card is hidden (e.g., starts with 'X') it returns an all-zeros vector.
    """
    ranks = ['A','2','3','4','5','6','7','8','9','10','J','Q','K']
    suits = ['S','H','D','C']
    onehot = np.zeros(52)
    if card is None or card.upper().startswith("X"):
        return onehot
    suit = card[0]
    rank = card[1:]
    if suit in suits and rank in ranks:
        suit_index = suits.index(suit)
        rank_index = ranks.index(rank)
        index = suit_index * 13 + rank_index
        onehot[index] = 1
    return onehot

def encode_board_cards(board_cards, max_cards=5):
    """
    Encode the board cards as the concatenation of one-hot vectors (52 dims each).
    Pads with zeros if there are fewer than max_cards.
    """
    encoded = []
    for card in board_cards:
        encoded.append(card_to_onehot(card))
    while len(encoded) < max_cards:
        encoded.append(np.zeros(52))
    return np.concatenate(encoded[:max_cards])

def encode_hole_cards(hole_cards):
    """
    Encode the player's (or actor's) hole cards (expected to be a list of 2 cards)
    as a concatenation of two 52-dim one-hot vectors.
    """
    encoded = []
    for card in hole_cards:
        encoded.append(card_to_onehot(card))
    while len(encoded) < 2:
        encoded.append(np.zeros(52))
    return np.concatenate(encoded[:2])

# =============================================================================
# 2. Process Each Snapshot into Model Inputs and Target
# =============================================================================

def process_snapshot(snapshot):
    """
    From a snapshot dictionary, create:
      - A vector of static features
      - A sequence of previous actions (each with an action type and amount)
      - The target (opponent's current action type)
      
    **Static features include:**
      - One-hot encoded round (4 dims)
      - Pot size (1 dim; scaled)
      - Blinds: small, big, ante (3 dims; scaled)
      - Actor stack (1 dim; scaled)
      - Actor position (one-hot, 10 dims)
      - Number of players remaining (1 dim; scaled)
      - Board cards (5 fixed cards × 52 dims = 260 dims)
      - Actor hole cards (2 cards × 52 dims = 104 dims)
      
    **Sequential features:**  
      For each previous action (up to a fixed max length) we use:
      - action_type (integer; offset by +1 so that 0 is reserved for padding)
      - action_sum (float; scaled)
    """
    # --- Static features ---
    # 1. Round (from current action)
    round_no = int(snapshot["action"]["round"])
    round_vec = one_hot_round(round_no)
    
    # 2. Pot size (scale by 100)
    pot_size = np.array([float(snapshot["pot_size"]) / 100.0])
    
    # 3. Blinds and ante (scaled)
    blinds = snapshot.get("blinds", {})
    small_blind = float(blinds.get("small_blind", 0)) / 100.0
    big_blind = float(blinds.get("big_blind", 0)) / 100.0
    ante = float(blinds.get("ante", 0)) / 100.0
    blinds_vec = np.array([small_blind, big_blind, ante])
    
    # 4. Actor stack size (scale by 1000)
    actor_stack = np.array([float(snapshot.get("actor_stack_size", 0)) / 1000.0])
    
    # 5. Actor position (one-hot with dimension 10)
    actor_pos = int(snapshot.get("actor_position", 0))
    pos_vec = one_hot_position(actor_pos, max_players=10)
    
    # 6. Number of players remaining (scale by 10)
    players_remaining = np.array([float(snapshot.get("players_remaining", 0)) / 10.0])
    
    # 7. Board cards (5 fixed cards)
    board_vec = encode_board_cards(snapshot.get("board_cards", []), max_cards=5)
    
    # 8. Actor hole cards (2 cards)
    hole_cards_vec = encode_hole_cards(snapshot.get("actor_hole_cards", []))
    
    # Concatenate all static features:
    # Total dims: 4 + 1 + 3 + 1 + 10 + 1 + 260 + 104 = 384
    static_features = np.concatenate([
        round_vec, pot_size, blinds_vec, actor_stack, pos_vec,
        players_remaining, board_vec, hole_cards_vec
    ])
    
    # --- Sequential features ---
    # For each previous action, we take:
    #   - action_type (offset by +1 so that 0 is our pad value)
    #   - action_sum (scaled by 100)
    seq_actions = snapshot.get("previous_actions", [])
    seq_types = []
    seq_amounts = []
    for action in seq_actions:
        act_type = int(action.get("action_type", 0)) + 1  # reserve 0 for padding
        act_sum = float(action.get("action_sum", 0)) / 100.0
        seq_types.append(act_type)
        seq_amounts.append(act_sum)
    
    MAX_SEQ_LENGTH = 20  # maximum number of previous actions to consider
    # Truncate if too long
    seq_types = seq_types[:MAX_SEQ_LENGTH]
    seq_amounts = seq_amounts[:MAX_SEQ_LENGTH]
    # Pad sequences (pad type=0, which for action_type will be masked in the Embedding layer)
    while len(seq_types) < MAX_SEQ_LENGTH:
        seq_types.append(0)
        seq_amounts.append(0.0)
    seq_types = np.array(seq_types, dtype=np.int32)
    seq_amounts = np.array(seq_amounts, dtype=np.float32).reshape((MAX_SEQ_LENGTH, 1))
    
    # --- Target: Opponent's current action type (as integer) ---
    target = int(snapshot["action"].get("action_type", 0))
    
    return static_features, seq_types, seq_amounts, target

# =============================================================================
# 3. Load and Preprocess Data
# =============================================================================

def load_and_preprocess_data(json_filename):
    """
    Load snapshot logs from a JSON file and create training arrays.
    The JSON file is expected to be a list of snapshots.
    """
    with open(json_filename, 'r') as f:
        data = json.load(f)
    
    static_features_list = []
    seq_types_list = []
    seq_amounts_list = []
    targets = []
    
    for snapshot in data:
        static_feat, seq_types, seq_amounts, target = process_snapshot(snapshot)
        static_features_list.append(static_feat)
        seq_types_list.append(seq_types)
        seq_amounts_list.append(seq_amounts)
        targets.append(target)
    
    X_static = np.stack(static_features_list)        # shape: (N, 384)
    X_seq_types = np.stack(seq_types_list)             # shape: (N, MAX_SEQ_LENGTH)
    X_seq_amounts = np.stack(seq_amounts_list)         # shape: (N, MAX_SEQ_LENGTH, 1)
    y = np.array(targets, dtype=np.int32)              # shape: (N,)
    
    return X_static, X_seq_types, X_seq_amounts, y

# Change the filename below to your JSON file produced by your XML parser.
JSON_FILENAME = 'logs.json'
X_static, X_seq_types, X_seq_amounts, y = load_and_preprocess_data(JSON_FILENAME)

# (Optional) Check the shapes of your training arrays:
print("X_static shape:", X_static.shape)
print("X_seq_types shape:", X_seq_types.shape)
print("X_seq_amounts shape:", X_seq_amounts.shape)
print("y shape:", y.shape)

# Split into training and validation sets
X_static_train, X_static_val, X_seq_types_train, X_seq_types_val, X_seq_amounts_train, X_seq_amounts_val, y_train, y_val = train_test_split(
    X_static, X_seq_types, X_seq_amounts, y, test_size=0.2, random_state=42
)

# =============================================================================
# 4. Build the Keras Model
# =============================================================================

# Parameters for the sequential branch
MAX_SEQ_LENGTH = 20
# Adjust NUM_ACTION_TYPES based on your data (here we assume 25; update if needed)
NUM_ACTION_TYPES = 25  
EMBEDDING_DIM = 16  
NUM_ACTION_CLASSES = 30  # number of distinct action types to predict (update as needed)

# -- Static Input Branch --
static_input = Input(shape=(384,), name='static_input')
x_static = Dense(128, activation='relu')(static_input)
x_static = Dense(64, activation='relu')(x_static)

# -- Sequential Input Branch --
# Input for action types (integers; shape = (MAX_SEQ_LENGTH,))
seq_types_input = Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name='seq_types_input')
# Input for action amounts (floats; shape = (MAX_SEQ_LENGTH, 1))
seq_amounts_input = Input(shape=(MAX_SEQ_LENGTH, 1), dtype='float32', name='seq_amounts_input')

# Process action types with an Embedding layer.
# (We use mask_zero=True so that padded 0 values are ignored.)
x_seq_types = Embedding(
    input_dim=NUM_ACTION_TYPES + 1,  # +1 to reserve index 0 for padding
    output_dim=EMBEDDING_DIM,
    mask_zero=True,
    name='action_type_embedding'
)(seq_types_input)

# Process the amounts with a simple dense layer (applied to each time step).
x_seq_amounts = Dense(8, activation='relu', name='amount_dense')(seq_amounts_input)

# Concatenate along the feature dimension: now each time step has (EMBEDDING_DIM + 8) features.
x_seq = Concatenate(name='seq_concat')([x_seq_types, x_seq_amounts])

# Process the concatenated sequence with an LSTM.
x_seq = LSTM(64, name='lstm_seq')(x_seq)

# -- Merge Both Branches --
x = Concatenate(name='merge')([x_static, x_seq])
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(NUM_ACTION_CLASSES, activation='softmax', name='output')(x)

model = Model(
    inputs=[static_input, seq_types_input, seq_amounts_input],
    outputs=output
)

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# =============================================================================
# 5. Train the Model
# =============================================================================

history = model.fit(
    x={
        'static_input': X_static_train,
        'seq_types_input': X_seq_types_train,
        'seq_amounts_input': X_seq_amounts_train
    },
    y=y_train,
    validation_data=(
        {
            'static_input': X_static_val,
            'seq_types_input': X_seq_types_val,
            'seq_amounts_input': X_seq_amounts_val
        },
        y_val
    ),
    epochs=20,
    batch_size=32
)

# =============================================================================
# 6. Evaluate / Save the Model
# =============================================================================

loss, acc = model.evaluate(
    x={
        'static_input': X_static_val,
        'seq_types_input': X_seq_types_val,
        'seq_amounts_input': X_seq_amounts_val
    },
    y=y_val
)
print(f"Validation Loss: {loss:.4f}  |  Validation Accuracy: {acc:.4f}")

# Optionally, save your model:
model.save("opponent_model.h5")


c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:519: 

X_static shape: (498975, 384)
X_seq_types shape: (498975, 20)
X_seq_amounts shape: (498975, 20, 1)
y shape: (498975,)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
seq_types_input (InputLayer)    [(None, 20)]         0                                            
__________________________________________________________________________________________________
seq_amounts_input (InputLayer)  [(None, 20, 1)]      0                                            
__________________________________________

## Testing the Model

In [1]:
#!/usr/bin/env python3
import json
import numpy as np
from tensorflow.keras.models import load_model

# ---------------------------------------------------------------------------
# Helper Functions (same as used during training)
# ---------------------------------------------------------------------------

def one_hot_round(round_no):
    """One-hot encode round number (1=preflop, 2=flop, 3=turn, 4=river)."""
    vec = np.zeros(4)
    if 1 <= round_no <= 4:
        vec[round_no - 1] = 1
    return vec

def one_hot_position(pos, max_players=10):
    """One-hot encode a player's position (an integer in [0, max_players-1])."""
    vec = np.zeros(max_players)
    if pos < max_players:
        vec[pos] = 1
    return vec

def card_to_onehot(card):
    """
    Convert a card string (e.g., 'S4', 'HA', 'C10') to a 52-dimensional one-hot vector.
    If the card is hidden (e.g., starts with 'X') or missing, return a zero vector.
    """
    ranks = ['A','2','3','4','5','6','7','8','9','10','J','Q','K']
    suits = ['S','H','D','C']
    onehot = np.zeros(52)
    if card is None or card.upper().startswith("X"):
        return onehot
    suit = card[0]
    rank = card[1:]
    if suit in suits and rank in ranks:
        suit_index = suits.index(suit)
        rank_index = ranks.index(rank)
        index = suit_index * 13 + rank_index
        onehot[index] = 1
    return onehot

def encode_board_cards(board_cards, max_cards=5):
    """
    Encode the board cards as the concatenation of one-hot vectors (each 52 dimensions).
    Pads with zeros if there are fewer than max_cards.
    """
    encoded = []
    for card in board_cards:
        encoded.append(card_to_onehot(card))
    while len(encoded) < max_cards:
        encoded.append(np.zeros(52))
    return np.concatenate(encoded[:max_cards])

def encode_hole_cards(hole_cards):
    """
    Encode the actor's hole cards (a list of 2 cards) as a concatenation of two 52-dim one-hot vectors.
    Pads with zeros if needed.
    """
    encoded = []
    for card in hole_cards:
        encoded.append(card_to_onehot(card))
    while len(encoded) < 2:
        encoded.append(np.zeros(52))
    return np.concatenate(encoded[:2])

def process_snapshot(snapshot):
    """
    Convert a snapshot dictionary into inputs for the model.
      - Static features (384-dimensional vector)
      - Sequential features: two sequences for action types and action amounts (length=20)
      - The target action type (as integer)
    """
    # ----- Static features -----
    round_no = int(snapshot["action"]["round"])
    round_vec = one_hot_round(round_no)
    
    # Scale pot size by 100
    pot_size = np.array([float(snapshot["pot_size"]) / 100.0])
    
    # Blinds and ante (scaled)
    blinds = snapshot.get("blinds", {})
    small_blind = float(blinds.get("small_blind", 0)) / 100.0
    big_blind = float(blinds.get("big_blind", 0)) / 100.0
    ante = float(blinds.get("ante", 0)) / 100.0
    blinds_vec = np.array([small_blind, big_blind, ante])
    
    # Actor stack (scaled by 1000)
    actor_stack = np.array([float(snapshot.get("actor_stack_size", 0)) / 1000.0])
    
    # Actor position as one-hot (dimension 10)
    actor_pos = int(snapshot.get("actor_position", 0))
    pos_vec = one_hot_position(actor_pos, max_players=10)
    
    # Number of players remaining (scaled by 10)
    players_remaining = np.array([float(snapshot.get("players_remaining", 0)) / 10.0])
    
    # Board cards: fixed 5 cards (each 52 dimensions)
    board_vec = encode_board_cards(snapshot.get("board_cards", []), max_cards=5)
    
    # Actor hole cards: 2 cards (each 52 dimensions)
    hole_cards_vec = encode_hole_cards(snapshot.get("actor_hole_cards", []))
    
    # Concatenate static features (4 + 1 + 3 + 1 + 10 + 1 + 260 + 104 = 384 dims)
    static_features = np.concatenate([
        round_vec, pot_size, blinds_vec, actor_stack, pos_vec,
        players_remaining, board_vec, hole_cards_vec
    ])
    
    # ----- Sequential features -----
    seq_actions = snapshot.get("previous_actions", [])
    seq_types = []
    seq_amounts = []
    for action in seq_actions:
        # Offset action type by +1 so that 0 is reserved for padding
        act_type = int(action.get("action_type", 0)) + 1  
        act_sum = float(action.get("action_sum", 0)) / 100.0
        seq_types.append(act_type)
        seq_amounts.append(act_sum)
    
    MAX_SEQ_LENGTH = 20
    seq_types = seq_types[:MAX_SEQ_LENGTH]
    seq_amounts = seq_amounts[:MAX_SEQ_LENGTH]
    # Pad the sequences if needed
    while len(seq_types) < MAX_SEQ_LENGTH:
        seq_types.append(0)
        seq_amounts.append(0.0)
    seq_types = np.array(seq_types, dtype=np.int32)
    seq_amounts = np.array(seq_amounts, dtype=np.float32).reshape((MAX_SEQ_LENGTH, 1))
    
    # ----- Target -----
    target = int(snapshot["action"].get("action_type", 0))
    
    return static_features, seq_types, seq_amounts, target

# ---------------------------------------------------------------------------
# Load the trained model
# ---------------------------------------------------------------------------

model = load_model("opponent_model.h5")
print("Model loaded successfully.")

# ---------------------------------------------------------------------------
# Load a test snapshot from logs.json
# ---------------------------------------------------------------------------

# Open your JSON file that contains the extracted snapshots
with open("test_hand.json", "r") as f:
    snapshots = json.load(f)

# For testing, we'll use the first snapshot in the file.
test_snapshot = snapshots[0]
print("Testing on the first snapshot from logs.json:")

# Optionally, print the snapshot to inspect it:
import pprint
pprint.pprint(test_snapshot)

# ---------------------------------------------------------------------------
# Process the test snapshot into model inputs
# ---------------------------------------------------------------------------

static_features, seq_types, seq_amounts, true_target = process_snapshot(test_snapshot)

# Reshape to add the batch dimension (for a single sample)
static_features = np.expand_dims(static_features, axis=0)    # Shape: (1, 384)
seq_types = np.expand_dims(seq_types, axis=0)                  # Shape: (1, 20)
seq_amounts = np.expand_dims(seq_amounts, axis=0)              # Shape: (1, 20, 1)

# ---------------------------------------------------------------------------
# Make a prediction using the model
# ---------------------------------------------------------------------------

predictions = model.predict({
    "static_input": static_features,
    "seq_types_input": seq_types,
    "seq_amounts_input": seq_amounts
})

# The output is a probability vector for each of the 30 action classes.
predicted_class = np.argmax(predictions, axis=1)[0]

print("\n--- Prediction Results ---")
print("Predicted action type:", predicted_class)
print("True action type (from snapshot):", true_target)
print("Prediction probabilities:\n", predictions[0])


c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:519: 

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorboard\compat\ten

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model loaded successfully.
Testing on the first snapshot from logs.json:
{'action': {'action_no': 2,
            'action_sum': 10.0,
            'action_type': 3,
            'player': 'GGM3M',
            'round': 1},
 'actor_hole_cards': ['C10', 'D5'],
 'actor_position': 0,
 'actor_stack_size': 500.0,
 'blinds': {'ante': 0.0, 'big_blind': 20.0, 'small_blind': 10.0},
 'board_cards': [],
 'current_street': 'preflop',
 'gamecode': '10136070425',
 'is_button': False,
 'player_positions': {'Bekocabron': 1, 'GGM3M': 0, 'mmpq8j6x8': 2},
 'player_stacks': {'Bekocabron': 500.0, 'GGM3M': 500.0, 'mmpq8j6x8': 500.0},
 'players_remaining': 2,
 'pot_size': 30.0,
 'previous_actions': [{'action_no': 1,
                       'action_sum': 0.0,
                       'action_type': 0,
                       'player': 'mmpq8j6x8',
                       'player_position': 2,
                       'round': 1}

## Training Model with bet-size group

In [ ]:
#!/usr/bin/env python3
import os
import xml.etree.ElementTree as ET
import json
import re
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, LSTM, Embedding, Concatenate, Dropout
)
from tensorflow.keras.optimizers import Adam

# ------------------------------
# 1. Helper Functions
# ------------------------------

def safe_float(text):
    try:
        return float(re.sub(r"[^\d\.]", "", text))
    except Exception:
        return 0.0

def street_from_round(round_no):
    return {1: "preflop", 2: "flop", 3: "turn", 4: "river"}.get(round_no, "unknown")

def get_hole_cards(game, player):
    for r in game.findall('round'):
        for elem in r.findall('cards'):
            if elem.attrib.get("type") == "Pocket" and elem.attrib.get("player") == player:
                if elem.text:
                    cards = elem.text.split()
                    if any(c.upper().startswith("X") for c in cards):
                        return ["unknown", "unknown"]
                    else:
                        return cards
    return ["unknown", "unknown"]

# ------------------------------
# 2. XML Parsing & Snapshot Creation
# ------------------------------

def parse_decision_logs(root, hero):
    """
    Process each game in the XML and extract a snapshot for every opponent action
    that is one of the allowed types:
      - fold (0), call (3), check (4), and raise actions (bet (5), all-in (7), raise (23)).
    """
    logs = []
    for game in root.findall('game'):
        gamecode = game.attrib.get("gamecode", "")
        general = game.find('general')
        blinds = {}
        blinds['small_blind'] = safe_float(general.findtext('smallblind', default="0"))
        blinds['big_blind'] = safe_float(general.findtext('bigblind', default="0"))
        blinds['ante'] = safe_float(general.findtext('ante', default="0"))
        
        players_elem = general.find('players')
        player_positions = {}
        player_stacks = {}
        sorted_players = sorted(players_elem.findall('player'), key=lambda p: int(p.attrib.get('seat', 0)))
        names = [p.attrib['name'] for p in sorted_players]
        dealer = None
        for p in sorted_players:
            if p.attrib.get('dealer','0')=='1':
                dealer = p.attrib['name']
                break
        if dealer is None:
            dealer = names[0]
        dealer_index = names.index(dealer)
        order = names[dealer_index+1:] + names[:dealer_index+1]
        for i, name in enumerate(order):
            player_positions[name] = i
        for p in sorted_players:
            name = p.attrib['name']
            player_stacks[name] = safe_float(p.attrib.get('chips', "0"))
        
        active_players = {name: True for name in player_positions}
        pot_size = 0.0
        board_cards = []
        cumulative_actions = []
        snapshot_action_counter = 0

        for r in game.findall('round'):
            round_no = int(r.attrib.get('no', 0))
            for child in r:
                if child.tag == "cards":
                    if child.attrib.get("type") != "Pocket":
                        if child.text:
                            board_cards.extend(child.text.split())
                elif child.tag == "action":
                    action_details = {
                        'round': round_no,
                        'player': child.attrib.get('player'),
                        'action_type': int(child.attrib.get('type')),
                        'action_sum': safe_float(child.attrib.get('sum'))
                    }
                    # Only include opponent actions (not hero) and allowed types.
                    allowed_types = {0, 3, 4, 5, 7, 23}
                    if child.attrib.get('player') != hero and action_details['action_type'] in allowed_types:
                        snapshot_action_counter += 1
                        action_details['action_no'] = snapshot_action_counter
                        snapshot = {
                            "gamecode": gamecode,
                            "round_no": round_no,
                            "current_street": street_from_round(round_no),
                            "blinds": blinds,
                            "player_positions": player_positions,
                            "player_stacks": player_stacks,
                            "pot_size": pot_size,
                            "board_cards": board_cards.copy(),
                            "previous_actions": [act.copy() for act in cumulative_actions],
                            "action": action_details.copy(),
                            "players_remaining": sum(1 for v in active_players.values() if v),
                            "is_button": (player_positions.get(child.attrib.get('player')) == len(player_positions)-1),
                            "actor_hole_cards": get_hole_cards(game, child.attrib.get('player')),
                            "actor_stack_size": player_stacks.get(child.attrib.get('player'), 0.0),
                            "actor_position": player_positions.get(child.attrib.get('player'))
                        }
                        logs.append(snapshot)
                    cumulative_actions.append(action_details.copy())
                    pot_size += action_details['action_sum']
                    if action_details['action_type'] == 0:  # if fold, mark inactive
                        active_players[action_details['player']] = False
    return logs

# ------------------------------
# 3. Data Processing & Label Creation
# ------------------------------

def process_snapshot(snapshot):
    """
    From a snapshot dictionary, produce:
      - static_features (384-dim vector)
      - seq_types: sequence of previous action types (as integers, padded to MAX_SEQ_LENGTH)
      - seq_amounts: corresponding action amounts (floats, padded)
      - y_action: simplified action label (0: fold, 1: call, 2: check, 3: raise)
      - y_bet: bet size category (0 to 3) for raise actions; for non-raise actions, a dummy value (0)
    """
    # --- Static features ---
    round_no = int(snapshot["action"]["round"])
    # One-hot encode round (4 dims)
    def one_hot_round(round_no):
        vec = np.zeros(4)
        if 1 <= round_no <= 4:
            vec[round_no - 1] = 1
        return vec
    round_vec = one_hot_round(round_no)
    
    pot_size = np.array([float(snapshot["pot_size"]) / 100.0])
    blinds = snapshot.get("blinds", {})
    small_blind = float(blinds.get("small_blind", 0)) / 100.0
    big_blind = float(blinds.get("big_blind", 0)) / 100.0
    ante = float(blinds.get("ante", 0)) / 100.0
    blinds_vec = np.array([small_blind, big_blind, ante])
    actor_stack = np.array([float(snapshot.get("actor_stack_size", 0)) / 1000.0])
    def one_hot_position(pos, max_players=10):
        vec = np.zeros(max_players)
        if pos < max_players:
            vec[pos] = 1
        return vec
    actor_pos = int(snapshot.get("actor_position", 0))
    pos_vec = one_hot_position(actor_pos, max_players=10)
    players_remaining = np.array([float(snapshot.get("players_remaining", 0)) / 10.0])
    
    # Board cards (each card is encoded as a 52-dim one-hot vector)
    def card_to_onehot(card):
        ranks = ['A','2','3','4','5','6','7','8','9','10','J','Q','K']
        suits = ['S','H','D','C']
        onehot = np.zeros(52)
        if card is None or card.upper().startswith("X"):
            return onehot
        suit = card[0]
        rank = card[1:]
        if suit in suits and rank in ranks:
            suit_index = suits.index(suit)
            rank_index = ranks.index(rank)
            index = suit_index * 13 + rank_index
            onehot[index] = 1
        return onehot
    
    def encode_board_cards(board_cards, max_cards=5):
        encoded = []
        for card in board_cards:
            encoded.append(card_to_onehot(card))
        while len(encoded) < max_cards:
            encoded.append(np.zeros(52))
        return np.concatenate(encoded[:max_cards])
    
    board_vec = encode_board_cards(snapshot.get("board_cards", []), max_cards=5)
    
    # Actor hole cards (2 cards)
    def encode_hole_cards(hole_cards):
        encoded = []
        for card in hole_cards:
            encoded.append(card_to_onehot(card))
        while len(encoded) < 2:
            encoded.append(np.zeros(52))
        return np.concatenate(encoded[:2])
    
    hole_cards_vec = encode_hole_cards(snapshot.get("actor_hole_cards", []))
    
    static_features = np.concatenate([
        round_vec, pot_size, blinds_vec, actor_stack, pos_vec,
        players_remaining, board_vec, hole_cards_vec
    ])
    
    # --- Sequential features ---
    seq_actions = snapshot.get("previous_actions", [])
    seq_types = []
    seq_amounts = []
    for action in seq_actions:
        act_type = int(action.get("action_type", 0)) + 1  # offset so 0 is padding
        act_sum = float(action.get("action_sum", 0)) / 100.0
        seq_types.append(act_type)
        seq_amounts.append(act_sum)
    
    MAX_SEQ_LENGTH = 20
    seq_types = seq_types[:MAX_SEQ_LENGTH]
    seq_amounts = seq_amounts[:MAX_SEQ_LENGTH]
    while len(seq_types) < MAX_SEQ_LENGTH:
        seq_types.append(0)
        seq_amounts.append(0.0)
    seq_types = np.array(seq_types, dtype=np.int32)
    seq_amounts = np.array(seq_amounts, dtype=np.float32).reshape((MAX_SEQ_LENGTH, 1))
    
    # --- Create Targets ---
    raw_action_type = int(snapshot["action"].get("action_type", 0))
    raw_action_sum = safe_float(snapshot["action"].get("sum"))
    # Map the raw action into one of four simplified actions.
    # (0: fold, 1: call, 2: check, 3: raise)
    if raw_action_type == 0:
        main_action = 0  # fold
    elif raw_action_type == 3:
        main_action = 1  # call
    elif raw_action_type == 4:
        main_action = 2  # check
    elif raw_action_type in {5, 7, 23}:
        main_action = 3  # raise
    else:
        return None  # skip if not one of the allowed types

    # For raise actions only, compute the bet size category.
    # For non-raise actions, we set the category to 0 (and later mask its loss).
    bet_size_category = 0
    if main_action == 3:
        if round_no == 1:  # preflop: use big-blind multiples
            bb = float(blinds.get("big_blind", 1))
            raise_ratio = raw_action_sum / bb if bb != 0 else 0
            if 2.0 <= raise_ratio < 2.3:
                bet_size_category = 0  # small raise
            elif 2.3 <= raise_ratio < 3.0:
                bet_size_category = 1  # mid raise
            elif 3.0 <= raise_ratio < 4.0:
                bet_size_category = 2  # big raise
            elif raise_ratio >= 4.0:
                bet_size_category = 3  # all in
            else:
                bet_size_category = 0
        else:  # postflop: use percentage of the pot
            pot = float(snapshot.get("pot_size", 1))
            bet_ratio = raw_action_sum / pot if pot != 0 else 0
            if bet_ratio < 0.35:
                bet_size_category = 0  # small bet
            elif 0.35 <= bet_ratio < 0.70:
                bet_size_category = 1  # mid bet
            elif 0.70 <= bet_ratio < 1.10:
                bet_size_category = 2  # big bet
            elif bet_ratio >= 1.10:
                bet_size_category = 3  # all in
            else:
                bet_size_category = 0

    return static_features, seq_types, seq_amounts, main_action, bet_size_category

def load_and_preprocess_data(json_filename):
    """
    Load the snapshot logs from a JSON file and produce:
      - X_static, X_seq_types, X_seq_amounts (the inputs)
      - y_action: the simplified action label (0,1,2,3)
      - y_bet: the bet size category (0-3) (only valid for raise actions)
      - bet_sample_weights: weight 1 for raise actions, 0 for others (so the bet loss is only computed when applicable)
    """
    with open(json_filename, 'r') as f:
        data = json.load(f)
    
    static_features_list = []
    seq_types_list = []
    seq_amounts_list = []
    y_action_list = []
    y_bet_list = []
    bet_sample_weights = []
    
    for snapshot in data:
        processed = process_snapshot(snapshot)
        if processed is None:
            continue
        static_feat, seq_types, seq_amounts, action_label, bet_label = processed
        static_features_list.append(static_feat)
        seq_types_list.append(seq_types)
        seq_amounts_list.append(seq_amounts)
        y_action_list.append(action_label)
        y_bet_list.append(bet_label)
        # Only for raise actions (action_label==3) do we want to learn bet size.
        if action_label == 3:
            bet_sample_weights.append(1.0)
        else:
            bet_sample_weights.append(0.0)
    
    X_static = np.stack(static_features_list)
    X_seq_types = np.stack(seq_types_list)
    X_seq_amounts = np.stack(seq_amounts_list)
    y_action = np.array(y_action_list, dtype=np.int32)
    y_bet = np.array(y_bet_list, dtype=np.int32)
    bet_sample_weights = np.array(bet_sample_weights, dtype=np.float32)
    
    return X_static, X_seq_types, X_seq_amounts, y_action, y_bet, bet_sample_weights

# ------------------------------
# 4. Keras Model Definition
# ------------------------------

# Parameters for the sequential branch
MAX_SEQ_LENGTH = 20
NUM_ACTION_TYPES = 25  # Adjust if needed.
EMBEDDING_DIM = 16  

# Static input branch
static_input = Input(shape=(384,), name='static_input')
x_static = Dense(128, activation='relu')(static_input)
x_static = Dense(64, activation='relu')(x_static)

# Sequential branch inputs
seq_types_input = Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name='seq_types_input')
seq_amounts_input = Input(shape=(MAX_SEQ_LENGTH, 1), dtype='float32', name='seq_amounts_input')

# Process action types via Embedding (with mask_zero so that padding is ignored)
x_seq_types = Embedding(
    input_dim=NUM_ACTION_TYPES + 1,
    output_dim=EMBEDDING_DIM,
    mask_zero=True,
    name='action_type_embedding'
)(seq_types_input)
x_seq_amounts = Dense(8, activation='relu', name='amount_dense')(seq_amounts_input)
x_seq = Concatenate(name='seq_concat')([x_seq_types, x_seq_amounts])
x_seq = LSTM(64, name='lstm_seq')(x_seq)

# Merge branches
merged = Concatenate(name='merge')([x_static, x_seq])
x = Dense(64, activation='relu')(merged)
x = Dropout(0.5)(x)

# Two outputs:
# (a) Main action: 4 classes [fold, call, check, raise]
action_output = Dense(4, activation='softmax', name='action_output')(x)
# (b) Bet size (for raise actions): 4 classes now (small, mid, big, all in)
bet_output = Dense(4, activation='softmax', name='bet_output')(x)

model = Model(
    inputs=[static_input, seq_types_input, seq_amounts_input],
    outputs=[action_output, bet_output]
)

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss={
        'action_output': 'sparse_categorical_crossentropy',
        'bet_output': 'sparse_categorical_crossentropy'
    },
    loss_weights={'action_output': 1.0, 'bet_output': 1.0},
    metrics=['accuracy'],
    weighted_metrics=[]  # This silences the warning.
)

model.summary()

# ------------------------------
# 5. Load Data & Train the Model
# ------------------------------

# Change JSON_FILENAME to the path of your logs.json produced by your XML parser.
JSON_FILENAME = 'logs.json'
X_static, X_seq_types, X_seq_amounts, y_action, y_bet, bet_sample_weights = load_and_preprocess_data(JSON_FILENAME)

print("X_static shape:", X_static.shape)
print("X_seq_types shape:", X_seq_types.shape)
print("X_seq_amounts shape:", X_seq_amounts.shape)
print("y_action shape:", y_action.shape)
print("y_bet shape:", y_bet.shape)

# Split into training and validation sets.
(X_static_train, X_static_val,
 X_seq_types_train, X_seq_types_val,
 X_seq_amounts_train, X_seq_amounts_val,
 y_action_train, y_action_val,
 y_bet_train, y_bet_val,
 bet_sample_weights_train, bet_sample_weights_val) = train_test_split(
    X_static, X_seq_types, X_seq_amounts, y_action, y_bet, bet_sample_weights,
    test_size=0.2, random_state=42
)

history = model.fit(
    x={
        'static_input': X_static_train,
        'seq_types_input': X_seq_types_train,
        'seq_amounts_input': X_seq_amounts_train
    },
    y={
        'action_output': y_action_train,
        'bet_output': y_bet_train
    },
    sample_weight={
        'action_output': np.ones_like(y_action_train, dtype=np.float32),
        'bet_output': bet_sample_weights_train
    },
    validation_data=(
        {
            'static_input': X_static_val,
            'seq_types_input': X_seq_types_val,
            'seq_amounts_input': X_seq_amounts_val
        },
        {
            'action_output': y_action_val,
            'bet_output': y_bet_val
        },
        {
            'action_output': np.ones_like(y_action_val, dtype=np.float32),
            'bet_output': bet_sample_weights_val
        }
    ),
    epochs=20,
    batch_size=32
)

loss, action_loss, bet_loss, action_acc, bet_acc = model.evaluate(
    x={
        'static_input': X_static_val,
        'seq_types_input': X_seq_types_val,
        'seq_amounts_input': X_seq_amounts_val
    },
    y={
        'action_output': y_action_val,
        'bet_output': y_bet_val
    },
    sample_weight={
        'action_output': np.ones_like(y_action_val, dtype=np.float32),
        'bet_output': bet_sample_weights_val
    }
)
print(f"Validation Loss: {loss:.4f}  |  Action Accuracy: {action_acc:.4f}  |  Bet Accuracy: {bet_acc:.4f}")

# Save the trained model.
model.save("opponent_model.h5")


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 seq_types_input (InputLayer)   [(None, 20)]         0           []                               
                                                                                                  
 seq_amounts_input (InputLayer)  [(None, 20, 1)]     0           []                               
                                                                                                  
 static_input (InputLayer)      [(None, 384)]        0           []                               
                                                                                                  
 action_type_embedding (Embeddi  (None, 20, 16)      416         ['seq_types_input[0][0]']        
 ng)                                                                                          

2025-02-19 18:10:54.406899: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


17883/17888 [============================>.] - ETA: 0s - loss: 0.8247 - action_output_loss: 0.8222 - bet_output_loss: 0.0025 - action_output_accuracy: 0.5961 - bet_output_accuracy: 0.9992WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


17888/17888 [==============================] - 92s 5ms/step - loss: 0.8247 - action_output_loss: 0.8222 - bet_output_loss: 0.0025 - action_output_accuracy: 0.5961 - bet_output_accuracy: 0.9992 - val_loss: 0.7141 - val_action_output_loss: 0.7141 - val_bet_output_loss: 4.5156e-07 - val_action_output_accuracy: 0.6523 - val_bet_output_accuracy: 1.0000
Epoch 2/20
17878/17888 [============================>.] - ETA: 0s - loss: 0.7161 - action_output_loss: 0.7161 - bet_output_loss: 1.8910e-06 - action_output_accuracy: 0.6611 - bet_output_accuracy: 1.0000WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


17888/17888 [==============================] - 92s 5ms/step - loss: 0.7161 - action_output_loss: 0.7161 - bet_output_loss: 1.8900e-06 - action_output_accuracy: 0.6611 - bet_output_accuracy: 1.0000 - val_loss: 0.6770 - val_action_output_loss: 0.6770 - val_bet_output_loss: 3.9868e-09 - val_action_output_accuracy: 0.6805 - val_bet_output_accuracy: 1.0000
Epoch 3/20
17888/17888 [==============================] - ETA: 0s - loss: 0.6924 - action_output_loss: 0.6924 - bet_output_loss: 4.6199e-08 - action_output_accuracy: 0.6750 - bet_output_accuracy: 1.0000WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


...

17888/17888 [==============================] - 91s 5ms/step - loss: 0.6426 - action_output_loss: 0.6426 - bet_output_loss: 4.0506e-10 - action_output_accuracy: 0.7007 - bet_output_accuracy: 1.0000 - val_loss: 0.6827 - val_action_output_loss: 0.6827 - val_bet_output_loss: 6.6642e-12 - val_action_output_accuracy: 0.6992 - val_bet_output_accuracy: 1.0000
Epoch 18/20
17882/17888 [============================>.] - ETA: 0s - loss: 0.6452 - action_output_loss: 0.6452 - bet_output_loss: 3.4915e-10 - action_output_accuracy: 0.7000 - bet_output_accuracy: 1.0000WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


17888/17888 [==============================] - 90s 5ms/step - loss: 0.6453 - action_output_loss: 0.6453 - bet_output_loss: 3.4904e-10 - action_output_accuracy: 0.7000 - bet_output_accuracy: 1.0000 - val_loss: 0.6890 - val_action_output_loss: 0.6890 - val_bet_output_loss: 0.0000e+00 - val_action_output_accuracy: 0.6996 - val_bet_output_accuracy: 1.0000
Epoch 19/20
17887/17888 [============================>.] - ETA: 0s - loss: 0.6413 - action_output_loss: 0.6413 - bet_output_loss: 3.6905e-10 - action_output_accuracy: 0.7018 - bet_output_accuracy: 1.0000WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


17888/17888 [==============================] - 91s 5ms/step - loss: 0.6413 - action_output_loss: 0.6413 - bet_output_loss: 3.6903e-10 - action_output_accuracy: 0.7018 - bet_output_accuracy: 1.0000 - val_loss: 0.7275 - val_action_output_loss: 0.7275 - val_bet_output_loss: 8.3303e-13 - val_action_output_accuracy: 0.6865 - val_bet_output_accuracy: 1.0000
Epoch 20/20
17883/17888 [============================>.] - ETA: 0s - loss: 0.6415 - action_output_loss: 0.6415 - bet_output_loss: 3.4559e-10 - action_output_accuracy: 0.7010 - bet_output_accuracy: 1.0000WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


17888/17888 [==============================] - 91s 5ms/step - loss: 0.6415 - action_output_loss: 0.6415 - bet_output_loss: 3.4550e-10 - action_output_accuracy: 0.7010 - bet_output_accuracy: 1.0000 - val_loss: 0.7260 - val_action_output_loss: 0.7260 - val_bet_output_loss: 0.0000e+00 - val_action_output_accuracy: 0.6942 - val_bet_output_accuracy: 1.0000


4472/4472 [==============================] - 6s 1ms/step - loss: 0.7260 - action_output_loss: 0.7260 - bet_output_loss: 0.0000e+00 - action_output_accuracy: 0.6942 - bet_output_accuracy: 1.0000
Validation Loss: 0.7260  |  Action Accuracy: 0.6942  |  Bet Accuracy: 1.0000


In [10]:
from tensorflow.keras.models import Model, load_model
# For example, load the model and make predictions on new data.
loaded_model = load_model("opponent_model.h5")
# We'll predict on the first 5 samples from the validation set.
predictions = loaded_model.predict({
    'static_input': X_static_val[:15],
    'seq_types_input': X_seq_types_val[:15],
    'seq_amounts_input': X_seq_amounts_val[:15]
})
action_preds, bet_preds = predictions
import numpy as np

# Define human-readable labels for the outputs.
action_labels = {0: "fold", 1: "call", 2: "check", 3: "raise"}
bet_labels = {0: "small", 1: "mid", 2: "big", 3: "all in"}

print("Sample Predictions:")
for i in range(10):
    # Get the probabilities for the current sample.
    action_prob = action_preds[i]
    bet_prob = bet_preds[i]
    
    # Determine the predicted classes by taking the argmax.
    action_class = np.argmax(action_prob)
    bet_class = np.argmax(bet_prob)
    
    # Format the probabilities (rounding for easier reading).
    formatted_action_prob = np.round(action_prob, 3)
    formatted_bet_prob = np.round(bet_prob, 3)
    
    # Print a more human-friendly message.
    print(f"\nSample {i+1}:")
    print(f"  Predicted Action: {action_labels[action_class]} (Probability: {formatted_action_prob[action_class]})")
    print(f"    Full Action Probabilities: {formatted_action_prob}")
    print(f"  Predicted Bet: {bet_labels[bet_class]} (Probability: {formatted_bet_prob[bet_class]})")
    print(f"    Full Bet Probabilities: {formatted_bet_prob}")

1/1 [==============================] - 0s 394ms/step
Sample Predictions:

Sample 1:
  Predicted Action: raise (Probability: 0.5820000171661377)
    Full Action Probabilities: [0.    0.    0.418 0.582]
  Predicted Bet: small (Probability: 1.0)
    Full Bet Probabilities: [1. 0. 0. 0.]

Sample 2:
  Predicted Action: check (Probability: 0.6990000009536743)
    Full Action Probabilities: [0.    0.    0.699 0.301]
  Predicted Bet: small (Probability: 1.0)
    Full Bet Probabilities: [1. 0. 0. 0.]

Sample 3:
  Predicted Action: fold (Probability: 0.5109999775886536)
    Full Action Probabilities: [0.511 0.39  0.    0.098]
  Predicted Bet: small (Probability: 1.0)
    Full Bet Probabilities: [1. 0. 0. 0.]

Sample 4:
  Predicted Action: fold (Probability: 0.7089999914169312)
    Full Action Probabilities: [0.709 0.198 0.    0.093]
  Predicted Bet: small (Probability: 1.0)
    Full Bet Probabilities: [1. 0. 0. 0.]

Sample 5:
  Predicted Action: fold (Probability: 0.4410000145435333)
    Full Ac

In [7]:
#!/usr/bin/env python3
import os
import xml.etree.ElementTree as ET
import json
import re
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, LSTM, Embedding, Concatenate, Dropout
)
from tensorflow.keras.optimizers import Adam

# ------------------------------
# 1. Helper Functions
# ------------------------------

def safe_float(text):
    try:
        return float(re.sub(r"[^\d\.]", "", text))
    except Exception:
        return 0.0

def street_from_round(round_no):
    return {1: "preflop", 2: "flop", 3: "turn", 4: "river"}.get(round_no, "unknown")

def get_hole_cards(game, player):
    # (This function is still used for snapshot creation but its output is no longer included in static features.)
    for r in game.findall('round'):
        for elem in r.findall('cards'):
            if elem.attrib.get("type") == "Pocket" and elem.attrib.get("player") == player:
                if elem.text:
                    cards = elem.text.split()
                    if any(c.upper().startswith("X") for c in cards):
                        return ["unknown", "unknown"]
                    else:
                        return cards
    return ["unknown", "unknown"]

# ------------------------------
# 2. XML Parsing & Snapshot Creation
# ------------------------------

def parse_decision_logs(root, hero):
    """
    Process each game in the XML and extract a snapshot for every opponent action
    (allowed types: fold (0), call (3), check (4), and raise actions (5, 7, 23)).
    """
    logs = []
    for game in root.findall('game'):
        gamecode = game.attrib.get("gamecode", "")
        general = game.find('general')
        blinds = {
            'small_blind': safe_float(general.findtext('smallblind', default="0")),
            'big_blind': safe_float(general.findtext('bigblind', default="0")),
            'ante': safe_float(general.findtext('ante', default="0"))
        }
        
        players_elem = general.find('players')
        player_positions = {}
        player_stacks = {}
        sorted_players = sorted(players_elem.findall('player'), key=lambda p: int(p.attrib.get('seat', 0)))
        names = [p.attrib['name'] for p in sorted_players]
        dealer = None
        for p in sorted_players:
            if p.attrib.get('dealer','0')=='1':
                dealer = p.attrib['name']
                break
        if dealer is None:
            dealer = names[0]
        dealer_index = names.index(dealer)
        order = names[dealer_index+1:] + names[:dealer_index+1]
        for i, name in enumerate(order):
            player_positions[name] = i
        for p in sorted_players:
            name = p.attrib['name']
            player_stacks[name] = safe_float(p.attrib.get('chips', "0"))
        
        active_players = {name: True for name in player_positions}
        pot_size = 0.0
        board_cards = []
        cumulative_actions = []
        snapshot_action_counter = 0

        for r in game.findall('round'):
            round_no = int(r.attrib.get('no', 0))
            for child in r:
                if child.tag == "cards":
                    if child.attrib.get("type") != "Pocket":
                        if child.text:
                            board_cards.extend(child.text.split())
                elif child.tag == "action":
                    action_details = {
                        'round': round_no,
                        'player': child.attrib.get('player'),
                        'action_type': int(child.attrib.get('type')),
                        'action_sum': safe_float(child.attrib.get('sum'))
                    }
                    allowed_types = {0, 3, 4, 5, 7, 23}
                    if child.attrib.get('player') != hero and action_details['action_type'] in allowed_types:
                        snapshot_action_counter += 1
                        action_details['action_no'] = snapshot_action_counter
                        snapshot = {
                            "gamecode": gamecode,
                            "round_no": round_no,
                            "current_street": street_from_round(round_no),
                            "blinds": blinds,
                            "player_positions": player_positions,
                            "player_stacks": player_stacks,
                            "pot_size": pot_size,
                            "board_cards": board_cards.copy(),
                            "previous_actions": [act.copy() for act in cumulative_actions],
                            "action": action_details.copy(),
                            "players_remaining": sum(1 for v in active_players.values() if v),
                            "is_button": (player_positions.get(child.attrib.get('player')) == len(player_positions)-1),
                            "actor_hole_cards": get_hole_cards(game, child.attrib.get('player')),
                            "actor_stack_size": player_stacks.get(child.attrib.get('player'), 0.0),
                            "actor_position": player_positions.get(child.attrib.get('player'))
                        }
                        logs.append(snapshot)
                    cumulative_actions.append(action_details.copy())
                    pot_size += action_details['action_sum']
                    if action_details['action_type'] == 0:  # if fold, mark inactive
                        active_players[action_details['player']] = False
    return logs

def process_all_hand_history(root_folder, hero=None):
    all_logs = []
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith(".xml"):
                file_path = os.path.join(dirpath, filename)
                try:
                    tree = ET.parse(file_path)
                    root_xml = tree.getroot()
                    if hero is None:
                        session_general = root_xml.find('general')
                        if session_general is not None and session_general.find('nickname') is not None:
                            hero = session_general.find('nickname').text.strip()
                    logs = parse_decision_logs(root_xml, hero)
                    all_logs.extend(logs)
                    print(f"Processed file: {file_path} -> {len(logs)} snapshots.")
                except ET.ParseError as e:
                    print(f"Error parsing XML file: {file_path}", e)
    return all_logs

# ------------------------------
# 3. Data Processing & Label Creation
# ------------------------------

def process_snapshot(snapshot):
    """
    From a snapshot dictionary, produce:
      - static_features (without actor hole cards)
      - seq_types: sequence of previous action types (integers, padded to MAX_SEQ_LENGTH)
      - seq_amounts: corresponding action amounts (floats, padded)
      - y_action: simplified action label (0: fold, 1: call, 2: check, 3: raise)
      - y_bet: bet size category (0 to 3) for raise actions (dummy 0 for non-raise)
    """
    # --- Static features (actor hole cards removed) ---
    round_no = int(snapshot["action"]["round"])
    def one_hot_round(round_no):
        vec = np.zeros(4)
        if 1 <= round_no <= 4:
            vec[round_no - 1] = 1
        return vec
    round_vec = one_hot_round(round_no)
    
    pot_size = np.array([float(snapshot["pot_size"]) / 100.0])
    blinds = snapshot.get("blinds", {})
    small_blind = float(blinds.get("small_blind", 0)) / 100.0
    big_blind = float(blinds.get("big_blind", 0)) / 100.0
    ante = float(blinds.get("ante", 0)) / 100.0
    blinds_vec = np.array([small_blind, big_blind, ante])
    actor_stack = np.array([float(snapshot.get("actor_stack_size", 0)) / 1000.0])
    def one_hot_position(pos, max_players=10):
        vec = np.zeros(max_players)
        if pos < max_players:
            vec[pos] = 1
        return vec
    actor_pos = int(snapshot.get("actor_position", 0))
    pos_vec = one_hot_position(actor_pos, max_players=10)
    players_remaining = np.array([float(snapshot.get("players_remaining", 0)) / 10.0])
    
    # Board cards encoding (each card → 52-dim one-hot; pad to 5 cards)
    def card_to_onehot(card):
        ranks = ['A','2','3','4','5','6','7','8','9','10','J','Q','K']
        suits = ['S','H','D','C']
        onehot = np.zeros(52)
        if card is None or card.upper().startswith("X"):
            return onehot
        suit = card[0]
        rank = card[1:]
        if suit in suits and rank in ranks:
            suit_index = suits.index(suit)
            rank_index = ranks.index(rank)
            index = suit_index * 13 + rank_index
            onehot[index] = 1
        return onehot
    
    def encode_board_cards(board_cards, max_cards=5):
        encoded = [card_to_onehot(card) for card in board_cards]
        while len(encoded) < max_cards:
            encoded.append(np.zeros(52))
        return np.concatenate(encoded[:max_cards])
    
    board_vec = encode_board_cards(snapshot.get("board_cards", []), max_cards=5)
    
    # Static features now: round_vec, pot_size, blinds_vec, actor_stack, pos_vec, players_remaining, board_vec
    static_features = np.concatenate([
        round_vec, pot_size, blinds_vec, actor_stack, pos_vec,
        players_remaining, board_vec
    ])
    
    # --- Sequential features ---
    seq_actions = snapshot.get("previous_actions", [])
    seq_types = []
    seq_amounts = []
    for action in seq_actions:
        act_type = int(action.get("action_type", 0)) + 1  # offset so 0 is padding
        act_sum = float(action.get("action_sum", 0)) / 100.0
        seq_types.append(act_type)
        seq_amounts.append(act_sum)
    
    MAX_SEQ_LENGTH = 20
    seq_types = seq_types[:MAX_SEQ_LENGTH]
    seq_amounts = seq_amounts[:MAX_SEQ_LENGTH]
    while len(seq_types) < MAX_SEQ_LENGTH:
        seq_types.append(0)
        seq_amounts.append(0.0)
    seq_types = np.array(seq_types, dtype=np.int32)
    seq_amounts = np.array(seq_amounts, dtype=np.float32).reshape((MAX_SEQ_LENGTH, 1))
    
    # --- Create Targets ---
    raw_action_type = int(snapshot["action"].get("action_type", 0))
    raw_action_sum = safe_float(snapshot["action"].get("sum"))
    if raw_action_type == 0:
        main_action = 0  # fold
    elif raw_action_type == 3:
        main_action = 1  # call
    elif raw_action_type == 4:
        main_action = 2  # check
    elif raw_action_type in {5, 7, 23}:
        main_action = 3  # raise
    else:
        return None  # skip unhandled types

    # For raise actions only, compute bet size category; for non-raise, target remains dummy 0.
    bet_size_category = 0
    if main_action == 3:
        if round_no == 1:  # preflop: use big-blind multiples
            bb = float(blinds.get("big_blind", 1))
            raise_ratio = raw_action_sum / bb if bb != 0 else 0
            if 2.0 <= raise_ratio < 2.3:
                bet_size_category = 0  # small raise
            elif 2.3 <= raise_ratio < 3.0:
                bet_size_category = 1  # mid raise
            elif 3.0 <= raise_ratio < 4.0:
                bet_size_category = 2  # big raise
            elif raise_ratio >= 4.0:
                bet_size_category = 3  # all in
            else:
                bet_size_category = 0
        else:  # postflop: use percentage of the pot
            pot = float(snapshot.get("pot_size", 1))
            bet_ratio = raw_action_sum / pot if pot != 0 else 0
            if bet_ratio < 0.35:
                bet_size_category = 0  # small bet
            elif 0.35 <= bet_ratio < 0.70:
                bet_size_category = 1  # mid bet
            elif 0.70 <= bet_ratio < 1.10:
                bet_size_category = 2  # big bet
            elif bet_ratio >= 1.10:
                bet_size_category = 3  # all in
            else:
                bet_size_category = 0

    return static_features, seq_types, seq_amounts, main_action, bet_size_category

def load_and_preprocess_data(json_filename):
    """
    Loads snapshots from a JSON file and builds:
      - X_static, X_seq_types, X_seq_amounts (inputs)
      - y_action: simplified action label (0,1,2,3)
      - y_bet: bet size category (0-3, valid only for raises)
      - bet_sample_weights: weight=1 for raise actions; 0 for others.
    """
    with open(json_filename, 'r') as f:
        data = json.load(f)
    
    static_features_list = []
    seq_types_list = []
    seq_amounts_list = []
    y_action_list = []
    y_bet_list = []
    bet_sample_weights = []
    
    for snapshot in data:
        processed = process_snapshot(snapshot)
        if processed is None:
            continue
        static_feat, seq_types, seq_amounts, action_label, bet_label = processed
        static_features_list.append(static_feat)
        seq_types_list.append(seq_types)
        seq_amounts_list.append(seq_amounts)
        y_action_list.append(action_label)
        y_bet_list.append(bet_label)
        # Only for raise actions (action_label==3) do we train bet size; others get weight 0.
        bet_sample_weights.append(1.0 if action_label == 3 else 0.0)
    
    X_static = np.stack(static_features_list)
    X_seq_types = np.stack(seq_types_list)
    X_seq_amounts = np.stack(seq_amounts_list)
    y_action = np.array(y_action_list, dtype=np.int32)
    y_bet = np.array(y_bet_list, dtype=np.int32)
    bet_sample_weights = np.array(bet_sample_weights, dtype=np.float32)
    
    return X_static, X_seq_types, X_seq_amounts, y_action, y_bet, bet_sample_weights

# ------------------------------
# 4. Keras Model Definition
# ------------------------------

# Parameters for sequential branch
MAX_SEQ_LENGTH = 20
NUM_ACTION_TYPES = 25  # (if needed, adjust)
EMBEDDING_DIM = 16  

# Note: The static features no longer include actor hole cards so their dimension is 280.
static_input = Input(shape=(280,), name='static_input')
x_static = Dense(128, activation='relu')(static_input)
x_static = Dense(64, activation='relu')(x_static)

# Sequential branch inputs
seq_types_input = Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name='seq_types_input')
seq_amounts_input = Input(shape=(MAX_SEQ_LENGTH, 1), dtype='float32', name='seq_amounts_input')

# Process action types via an Embedding layer (with mask_zero so padding is ignored)
x_seq_types = Embedding(
    input_dim=NUM_ACTION_TYPES + 1,
    output_dim=EMBEDDING_DIM,
    mask_zero=True,
    name='action_type_embedding'
)(seq_types_input)
x_seq_amounts = Dense(8, activation='relu', name='amount_dense')(seq_amounts_input)
x_seq = Concatenate(name='seq_concat')([x_seq_types, x_seq_amounts])
x_seq = LSTM(64, name='lstm_seq')(x_seq)

# Merge static and sequential branches
merged = Concatenate(name='merge')([x_static, x_seq])
x = Dense(64, activation='relu')(merged)
x = Dropout(0.5)(x)

# Two outputs:
# (a) Main action: 4 classes [fold, call, check, raise]
action_output = Dense(4, activation='softmax', name='action_output')(x)
# (b) Bet size (only used when action is raise): 4 classes [small, mid, big, all in]
bet_output = Dense(4, activation='softmax', name='bet_output')(x)

model = Model(
    inputs=[static_input, seq_types_input, seq_amounts_input],
    outputs=[action_output, bet_output]
)

# In model compilation, we now provide a loss weight of 0 (via sample weights) for non-raise examples,
# and we remove the built‑in metric for bet_output (since its reported accuracy was misleading).
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss={
        'action_output': 'sparse_categorical_crossentropy',
        'bet_output': 'sparse_categorical_crossentropy'
    },
    loss_weights={'action_output': 1.0, 'bet_output': 1.0},
    metrics={'action_output': ['accuracy'], 'bet_output': []}
)

model.summary()

# ------------------------------
# 5. Main Execution: Data Extraction & Model Training
# ------------------------------

if __name__ == '__main__':
    # ----- (A) Data Extraction from XML Hand Histories -----

    
    # root_folder = "ipoker_hh_test"  # Change this to your folder containing XML files.
    # hero_name = None  # Or set your hero name manually (e.g., "mmpq8j6x8")
    
    # print("Processing all hand history XML files in folder:", root_folder)
    # all_logs = process_all_hand_history(root_folder, hero=hero_name)
    # print("Total snapshots extracted:", len(all_logs))
    
    # # Save snapshots to a JSON file (for later training)
    # with open("logs.json", "w") as outfile:
    #     json.dump(all_logs, outfile, indent=4)
    # print("Data extraction complete. Saved to logs.json")
    
    # ----- (B) Load Data & Train the Model -----
    JSON_FILENAME = 'logs.json'
    X_static, X_seq_types, X_seq_amounts, y_action, y_bet, bet_sample_weights = load_and_preprocess_data(JSON_FILENAME)
    
    print("X_static shape:", X_static.shape)
    print("X_seq_types shape:", X_seq_types.shape)
    print("X_seq_amounts shape:", X_seq_amounts.shape)
    print("y_action shape:", y_action.shape)
    print("y_bet shape:", y_bet.shape)
    
    # Split into training and validation sets.
    (X_static_train, X_static_val,
     X_seq_types_train, X_seq_types_val,
     X_seq_amounts_train, X_seq_amounts_val,
     y_action_train, y_action_val,
     y_bet_train, y_bet_val,
     bet_sample_weights_train, bet_sample_weights_val) = train_test_split(
        X_static, X_seq_types, X_seq_amounts, y_action, y_bet, bet_sample_weights,
        test_size=0.2, random_state=42
    )
    
    history = model.fit(
        x={
            'static_input': X_static_train,
            'seq_types_input': X_seq_types_train,
            'seq_amounts_input': X_seq_amounts_train
        },
        y={
            'action_output': y_action_train,
            'bet_output': y_bet_train
        },
        sample_weight={
            'action_output': np.ones_like(y_action_train, dtype=np.float32),
            'bet_output': bet_sample_weights_train  # only raise actions contribute here
        },
        validation_data=(
            {
                'static_input': X_static_val,
                'seq_types_input': X_seq_types_val,
                'seq_amounts_input': X_seq_amounts_val
            },
            {
                'action_output': y_action_val,
                'bet_output': y_bet_val
            },
            {
                'action_output': np.ones_like(y_action_val, dtype=np.float32),
                'bet_output': bet_sample_weights_val
            }
        ),
        epochs=2,
        batch_size=32
    )
    
    results = model.evaluate(
        x={
            'static_input': X_static_val,
            'seq_types_input': X_seq_types_val,
            'seq_amounts_input': X_seq_amounts_val
        },
        y={
            'action_output': y_action_val,
            'bet_output': y_bet_val
        },
        sample_weight={
            'action_output': np.ones_like(y_action_val, dtype=np.float32),
            'bet_output': bet_sample_weights_val
        }
    )
    print("Evaluation results:", results)
    
    # Save the trained model.
    model.save("opponent_model.h5")
    print("Model saved as opponent_model.h5")
    
    # IMPORTANT: At inference time, use the predicted bet_output only if the predicted main action (from action_output)
    # is 'raise' (i.e. class 3). Otherwise, ignore the bet prediction.


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 seq_types_input (InputLayer)   [(None, 20)]         0           []                               
                                                                                                  
 seq_amounts_input (InputLayer)  [(None, 20, 1)]     0           []                               
                                                                                                  
 static_input (InputLayer)      [(None, 280)]        0           []                               
                                                                                                  
 action_type_embedding (Embeddi  (None, 20, 16)      416         ['seq_types_input[0][0]']        
 ng)                                                                                        

17888/17888 [==============================] - 95s 5ms/step - loss: 0.8634 - action_output_loss: 0.8599 - bet_output_loss: 0.0035 - action_output_accuracy: 0.5642 - val_loss: 0.7912 - val_action_output_loss: 0.7912 - val_bet_output_loss: 8.4913e-08 - val_action_output_accuracy: 0.5931
Epoch 2/2
17880/17888 [============================>.] - ETA: 0s - loss: 0.7994 - action_output_loss: 0.7994 - bet_output_loss: 1.9826e-06 - action_output_accuracy: 0.5972WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


17888/17888 [==============================] - 92s 5ms/step - loss: 0.7994 - action_output_loss: 0.7994 - bet_output_loss: 1.9817e-06 - action_output_accuracy: 0.5972 - val_loss: 0.7815 - val_action_output_loss: 0.7815 - val_bet_output_loss: 7.3723e-10 - val_action_output_accuracy: 0.6046


4472/4472 [==============================] - 6s 1ms/step - loss: 0.7815 - action_output_loss: 0.7815 - bet_output_loss: 7.3723e-10 - action_output_accuracy: 0.6046
Evaluation results: [0.7815091609954834, 0.7815091609954834, 7.372268351346634e-10, 0.6046022176742554]
Model saved as opponent_model.h5
